In [1]:
import pandas as pd
import numpy as np
import warnings
import re
warnings.filterwarnings("ignore", category = DeprecationWarning)
#loading libraries 
import gensim
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim import models
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS
from gensim import corpora
from gensim.utils import simple_preprocess
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess

import pyLDAvis.gensim

import nltk
from nltk.probability import FreqDist

from nltk.corpus import stopwords
from nltk import word_tokenize
from spellchecker import SpellChecker
from nltk.stem import WordNetLemmatizer
from nltk import FreqDist
 
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS

import matplotlib.pyplot as plt
from wordcloud import WordCloud
import wordcloud

import spacy
from spacy.lang.en.stop_words import STOP_WORDS



In [2]:
df = pd.read_csv('G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\CASSIGNMENT-1.csv',encoding='unicode_escape')

In [3]:
df.columns

Index(['Sr No.', 'Name', 'Roll No', 'Gender', 'Email Address', 'Timestamp',
       'Find out errors(if any), write solution and also give explanation of the following code: \n#include <stdio.h>  \nint main()  \n{      \n           a = 10;      \n           printf("The value of a is : %d", a);     \n           return 0;  \n}  '],
      dtype='object')

In [4]:
df.columns=['No.','Name','ID','Gender','Email Address','Timestatmp','Comment1']

In [5]:
data=df.copy()

## <font color="Blue"> REMOVE ROW WITH BLANK CELL</font>

In [6]:
def removeblankrow(data,blank_row_info,col):
    
    print(f"Total number of row befor cleaning : {len(data)}")
    # Store the rows with empty lists in the specified column
    blank_rows = data[data[col].apply(lambda x: len(x) == 0)]
    
    # Extract the index, student ID, name, and gender of the blank rows
    blank_rows_info = blank_rows[['ID', 'Name', 'Gender','Comment1']]
    
    # Remove rows with empty lists in the specified column
    data = data[data[col].apply(lambda x: len(x) > 0)]
    
    # Reset index if needed
    data.reset_index(drop=True, inplace=True)
    
    print(f"\n Total Number of row after cleaning :{len(data)}") 
    print(f"\n Total Number of blank row is :{len(blank_rows_info)}")
    
    print(f"\n{blank_rows_info}")
    
    return data,blank_row_info

# Example usage:
# new_data, blank_rows_info = remove_blank_row(data, 'cleancomment3')
# print(blank_rows_info)


In [7]:
blankrow=pd.DataFrame()
data,blankrow=removeblankrow(data,blankrow,"Comment1")


Total number of row befor cleaning : 283

 Total Number of row after cleaning :283

 Total Number of blank row is :0

Empty DataFrame
Columns: [ID, Name, Gender, Comment1]
Index: []


## <font color="blue"> CLEANING DATASET </FONT>

In [8]:
c_keywords = [
    "auto", "break", "case", "char", "const", "continue", "default", "do", "double",
    "else", "enum", "extern", "float", "for", "goto", "if", "int", "long", "register",
    "return", "short", "signed", "sizeof", "static", "struct", "switch", "typedef",
    "union", "unsigned", "void", "volatile", "while"
]

In [9]:
# convert all comments in lower case
data['Comment1']=df['Comment1'].str.lower()

In [10]:
teacher_answer="Error: There is compile-time error. Variable is not declare. Solution: Variable must be declare before use. Here we have to declare it integer."

In [11]:
def clean_comment(comment):
    """Cleans student comments, removing C programming statements and optionally applying other preprocessing steps."""
    
    # Remove C programming statements
    comment = re.sub(r"#.*", "", comment) # Remove header files
    comment = re.sub(r"(void|int|char|float|double)\s+[\w_]+\s*\([^)]*\)[\s\{]", "", comment)  # Remove function declarations
    comment = re.sub(r"\{[\s\S]*?\}", "", comment)  # Remove code blocks within curly braces

    comment = re.sub(r"[\=+:\-*/%\(\)\{\}\[\];\"\'`<,>,.,0-9]", " ", comment)

   
#     for keyword in c_keywords:
#         comment = re.sub(rf"\b{keyword} ", "", comment) 
           
    comment = re.sub(r"\b\w{1,2}\b", "", comment)  # Remove single- or two-letter words
    comment = re.sub(r"\s+", " ", comment)  # Remove extra whitespace
    comment = re.sub("\S*\d\S*", "", comment).strip() # remove word which contain digit in it.


    return comment

<>:17: DeprecationWarning: invalid escape sequence '\S'
<>:17: DeprecationWarning: invalid escape sequence '\S'
C:\Users\Amit\AppData\Local\Temp\ipykernel_9084\2927420580.py:17: DeprecationWarning: invalid escape sequence '\S'
  comment = re.sub("\S*\d\S*", "", comment).strip() # remove word which contain digit in it.


In [12]:
data["CleanComment"]=data['Comment1'].apply(clean_comment)

In [13]:
teacher_answer=teacher_answer.lower()
clean_teacher_answer=clean_comment(teacher_answer)

In [14]:
clean_teacher_answer

'error there compile time error variable not declare solution variable must declare before use here have declare integer'

In [15]:
data,blankrow=removeblankrow(data,blankrow,"CleanComment")

Total number of row befor cleaning : 283

 Total Number of row after cleaning :266

 Total Number of blank row is :17

            ID                              Name  Gender  \
17    23BCA286             SIRVI VIKAS POKARRAM     Male   
26    23BCA107           Mistry sachi Umeshbhai   Female   
34    23BCA078             Lad Trisa Mukeshbhai   Female   
40    23BCA260           Reema Umeshbhai sahani   Female   
60    23bca258          RATHOD VICKY SHARADBHAI     Male   
72    23BCA316              YADAV SONAL RAMGYAN   Female   
101   23BCA034     Das Vihangkumar Abhijeetbhai     Male   
111   23BCA015           BHAVSAR PURV HIRENBHAI     Male   
124     BCA053               Tushar Anil Jadhav     Male   
177   23BCA023  KRINCEKUMAR NAVINBHAI CHAUDHARI     Male   
178   23bca005           Ahir kheyan Nileshbhai     Male   
242   23BCA077              Lad meet ishvarbhai     Male   
253  23BCA178                       Patel misha   Female   
261   23BCA315             Yadav shivam r

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   No.            266 non-null    int64 
 1   Name           266 non-null    object
 2   ID             266 non-null    object
 3   Gender         266 non-null    object
 4   Email Address  266 non-null    object
 5   Timestatmp     266 non-null    object
 6   Comment1       266 non-null    object
 7   CleanComment   266 non-null    object
dtypes: int64(1), object(7)
memory usage: 16.8+ KB


We find that from 283 there is 257 records remaining from the above output. 

so 283-257 = 26 students comments are remove after clearning the comment.

#### CleanComment : Contain the comment after removing stopwords, programming keywords, Programming Statement, Remove multiple spaces, digits etc..

### <font color="blue"> FIND SYNONYMES </font>

In [17]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Python310\lib\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [18]:
from nltk.corpus import wordnet
import pandas as pd

# Function to get synonyms of a word using WordNet
def get_synonyms(word):
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
    return synonyms

# Load your DataFrame 'data' containing comments in 'Cleancomment' column
# Assuming 'data' is already loaded

# Initialize lists to store synonyms of 'error' and 'solution'
error_synonyms_list = []
solution_synonyms_list = []

# Iterate over each comment in the DataFrame
for comment in data['CleanComment']:
    # Find synonyms of 'error' and 'solution' in the comment
    error_synonyms = get_synonyms('error')
    solution_synonyms = get_synonyms('solution')
    
    # Store synonyms in the lists
    error_synonyms_list.extend(error_synonyms)
    solution_synonyms_list.extend(solution_synonyms)

# Remove duplicates from the lists
error_synonyms_list = list(set(error_synonyms_list))
solution_synonyms_list = list(set(solution_synonyms_list))

# Print and store synonyms in a list
print("Synonyms of 'error':", error_synonyms_list)
print("Synonyms of 'solution':", solution_synonyms_list)

# Optionally, you can store the synonyms lists in your DataFrame or elsewhere for later use


Synonyms of 'error': ['wrongdoing', 'erroneous_belief', 'mistake', 'misplay', 'fault', 'erroneousness', 'computer_error', 'error']
Synonyms of 'solution': ['resolution', 'solution', 'answer', 'root', 'result', 'solvent']


In [19]:
data["CleanComment"][0]

'the code you provided missing declaration for the variable fix this error you should add line the beginning the main function that declares integer the corrected code would look like this this code declares integer and initializes with the value then prints the value using the printf function finally the main function returns indicate that the program has run successfully'

### <font color="blue"> REMOVE STOPWORDS </font>

In [20]:
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize

# Load English stopwords
stop_words = set(stopwords.words('english'))

# Function to remove stopwords and store them in a text file
def remove_stopwords_and_store(comment):
    # Tokenize the comment
    words = word_tokenize(comment)
    
    # Remove stopwords and store them in a set
    removed_stopwords = [word for word in words if word.lower() not in stop_words]
    
    # Convert removed stopwords to a string
    removed_stopwords_str = ' '.join(removed_stopwords)
    
    # Store removed stopwords in a text file
    with open('removed_stopwords.txt', 'a') as file:
        file.write(' '.join(set(words) - set(removed_stopwords)) + '\n')
    
    return removed_stopwords_str

# Apply the function to each comment in the dataframe and store the result in a new column
data["comment_without_stopword"] = data["CleanComment"].apply(remove_stopwords_and_store)
lem_teacher_answer=remove_stopwords_and_store(clean_teacher_answer)

In [21]:
data["comment_without_stopword"][0]

'code provided missing declaration variable fix error add line beginning main function declares integer corrected code would look like code declares integer initializes value prints value using printf function finally main function returns indicate program run successfully'

In [22]:
teacher_answer

'error: there is compile-time error. variable is not declare. solution: variable must be declare before use. here we have to declare it integer.'

In [23]:
lem_teacher_answer

'error compile time error variable declare solution variable must declare use declare integer'

In [24]:
# stop_words = stopwords.words('english')
# data["comment_without_stopword"] = data["CleanComment"].apply(lambda x:' '.join([word for word in word_tokenize(x) if word not in stop_words]))

In [25]:
data["Comment_Length"]=data["CleanComment"].apply(lambda x:len(x))

### <font color="blue"> APPLY BIGRAM TRIGRAM BEFORE REMOVE STOP WORDS </font>

In [26]:
import csv
import itertools

def find_similar_phrases(data):
    # Initialize a dictionary to store counts of word pairs
    word_pairs_count = {}
    
    # Iterate over each comment in the data
    for comment in data['comment_without_stopword']:
        # Split the comment into words
        words = comment.split()
        
        # Generate all pairs of adjacent words
        pairs = list(zip(words, words[1:]))
        
        # Update the word_pairs_count dictionary
        for pair in pairs:
            word_pairs_count[pair] = word_pairs_count.get(pair, 0) + 1
    
    # Filter pairs where both words are alphabetic and have a count greater than a threshold
    similar_phrases = [(pair[0], pair[1], count) for pair, count in word_pairs_count.items() if count > threshold
                       and pair[0].isalpha() and pair[1].isalpha()]
    
    return similar_phrases

# Set the threshold for considering word pairs
threshold = 10

# Find similar phrases in the data
similar_phrases = find_similar_phrases(data)

# Print the list of similar phrases in the desired format
for i, (word1, word2, count) in enumerate(similar_phrases, start=1):
    print(f"{i}. ({word1}, {word2}) - Count: {count}")

# Store the list of similar phrases and their occurrences in a CSV file
output_file = "G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\similar_phrases_with_counts.csv"
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Index", "Word1", "Word2", "Count"])
    for i, (word1, word2, count) in enumerate(similar_phrases, start=1):
        writer.writerow([i, word1, word2, count])

print("Similar phrases with counts have been stored in the CSV file:", output_file)

1. (main, function) - Count: 28
2. (declares, integer) - Count: 28
3. (corrected, code) - Count: 41
4. (initializes, value) - Count: 15
5. (prints, value) - Count: 25
6. (value, using) - Count: 40
7. (using, printf) - Count: 78
8. (printf, function) - Count: 27
9. (syntax, error) - Count: 21
10. (declaration, error) - Count: 12
11. (variable, used) - Count: 76
12. (used, without) - Count: 85
13. (without, declaration) - Count: 56
14. (errors, code) - Count: 12
15. (variable, declared) - Count: 51
16. (data, type) - Count: 101
17. (need, declare) - Count: 33
18. (declare, variable) - Count: 47
19. (assigning, value) - Count: 14
20. (error, variable) - Count: 41
21. (solution, int) - Count: 30
22. (explanation, added) - Count: 21
23. (added, int) - Count: 75
24. (int, declare) - Count: 75
25. (variable, integer) - Count: 30
26. (integer, initialized) - Count: 17
27. (initialized, value) - Count: 23
28. (value, printed) - Count: 18
29. (printed, value) - Count: 16
30. (printf, correct) - 

In [ ]:
import pandas as pd

def find_pairs_with_word(similar_phrases, target_word):
    pairs_containing_word = []
    
    # Find all pairs containing the target word
    for phrase_pair in similar_phrases:
        if target_word in phrase_pair:
            pairs_containing_word.append(phrase_pair)
    
    # Remove counts from the pairs
    pairs_containing_word = [(word1, word2) for word1, word2, _ in pairs_containing_word]
    
    return pairs_containing_word

def combine_phrases(pair):
    return ''.join(pair)

# Ask the user for the target word
target_word = input("Enter the word to search for: ")

# Find pairs containing the target word
pairs_containing_target_word = find_pairs_with_word(similar_phrases, target_word)

# Print the pairs containing the target word
print("Pairs containing '{}':".format(target_word))
for i, phrase_pair in enumerate(pairs_containing_target_word):
    print("{}. {}".format(i+1, ' '.join(phrase_pair)))

# If no pairs found, inform the user and exit
if not pairs_containing_target_word:
    print("No pairs found containing '{}'.".format(target_word))
else:
    # Ask the user for their choice of pair
    choice = input("Enter the number of the pair to combine: ")
    try:
        choice_index = int(choice) - 1
        chosen_pair = pairs_containing_target_word[choice_index]
        
        # Combine the chosen pair
        combined_phrase = combine_phrases(chosen_pair)
        print("Combined phrase:", combined_phrase)
        
        # Apply the combined phrase to the dataset
        data['apply_bigram_trigram'] = data['comment_without_stopword'].apply(lambda x: x.replace(' '.join(chosen_pair), combined_phrase))
        
        # Store the details where changes occurred
        combine_word = data[data['apply_bigram_trigram'].str.contains(combined_phrase)]
        combine_word.reset_index(drop=True, inplace=True)
        
        # Save the dataframe with combined words
        combine_word.to_csv('G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\combine_word_ass1.csv', index=False)
        
        # Save the updated original dataframe
#         data.to_csv('data_with_changes.csv', index=False)
        
        print("Changes applied and stored in separate dataframes successfully.")
    except (ValueError, IndexError):
        print("Invalid choice. Please enter a valid number.")

### <font color="blue"> VISUALIZE WORD FREQUENCY AFTER REMOVING STOPWORDS </font>

In [ ]:
tokenized_comments = [word_tokenize(comment) for comment in data["apply_bigram_trigram"]]

# Flatten the list of tokenized comments
flat_tokens = [token for sublist in tokenized_comments for token in sublist ] #if len(token)>4]

# Create a frequency distribution
freq_dist = FreqDist(flat_tokens)

# Display the most common words and their frequencies
print("Top 20 Most Common Words:")
print(freq_dist.most_common(50))

In [ ]:
data.rename(columns = {'apply_bigram_trigram':'comment_after_bitrigram'}, inplace = True)

In [ ]:
from matplotlib import pyplot as plt
#top_words = word_freq.most_common(20)
x, y = zip(*freq_dist.most_common(20) )
plt.figure(figsize=(10, 6))
plt.bar(x, y)
plt.title('Top 20 Most Common Words (Before Lemmatization)')
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### DICTIONARY (CORPUS) OF ALL UNIQUE WORD USED IN COMMENT

In [ ]:
import re

# Extract all unique words from the 'comment_after_bitrigram' column
unique_words = set()
for comment in data['comment_after_bitrigram']:
    # Split the comment into words and remove symbols and special characters
    words = re.findall(r'\b\w+\b', comment)
    unique_words.update(words)

# Convert the set of unique words to a list
unique_words_list = list(unique_words)

# Write the unique words to a text file with UTF-8 encoding
file_path = r'G:\GitHub Classroom Assistant\Topic Modelling Dataset\TMC\CORPUS\WORDS.TXT'
with open(file_path, 'w', encoding='utf-8') as file:
    for word in unique_words_list:
        file.write(word + '\n')

print("Total number of unique words:", len(unique_words_list))
print("Unique words have been written to:", file_path)



### <font color="blue"> SPELL CHECK USING "SPELLCHECKER" PACKAGES.</font>

In [ ]:
import pandas as pd
from spellchecker import SpellChecker

# Initialize SpellChecker with English language
spell = SpellChecker(language='en')

# Define words to exclude from correction
excluded_words = {'scanf', 'printf', 'getch', 'stdio', 'conio', 'clrscr', 'getchar',  
                  'clrscr', 'void', 'int', 'datatype','runtime'}

# Add excluded_words in spellchecker directory.

# Define manual corrections for specific words
manual_corrections = {'flot': 'float', 'lot':'float','clrsccr': 'clrscr','flost':'float','\x91clrscr':'clrscr'}

# Function to spellcheck each comment
def spellcheck_comment(comment):
    if comment is None or comment == "":
        return "", {}, 0

    # Tokenize the comment
    tokens = comment.split()

    # Initialize dictionary to store misspelled words and their corrections
    misspellings = {}

    # Iterate through each word in the comment
    for word in tokens:
        # Check if the word is in the excluded list
        if word.lower() in excluded_words:
            continue  # Skip correction for excluded words
        
        # Check if the word is misspelled
        corrected_word = spell.correction(word)
        
        # Check if the corrected word is one of the manual corrections
        if corrected_word in manual_corrections:
            corrected_word = manual_corrections[corrected_word]
        
        if corrected_word != word:
            # Store the misspelled word and its corrected form in the dictionary
            misspellings[word] = corrected_word

    # Replace misspelled words with correct spelling in the comment
    corrected_tokens = []
    total_incorrect_words = 0
    for word in tokens:
        corrected_word = misspellings.get(word)
        if corrected_word is not None:
            corrected_tokens.append(corrected_word)
            total_incorrect_words += 1
        else:
            corrected_tokens.append(word)
    
    corrected_comment = ' '.join(corrected_tokens)

    return corrected_comment, misspellings, total_incorrect_words

# Apply spellchecking to each comment in the dataframe
data['CorrectComment_Spell'], data['misspellings'], data['total_incorrect_words'] = zip(*data['comment_after_bitrigram'].apply(spellcheck_comment))

# Create a dictionary of incorrect spellings against correct spellings
incorrect_correct_dict = {}
for misspelling_dict in data['misspellings']:
    for misspelled_word, corrected_word in misspelling_dict.items():
        if misspelled_word not in incorrect_correct_dict:
            incorrect_correct_dict[misspelled_word] = corrected_word

# Create a new dataframe with required columns
new_data = data[['ID', 'Name', 'Comment1', 'comment_without_stopword', 'misspellings', 'CorrectComment_Spell', 'total_incorrect_words']]

# Display the corrected comments and misspelled words
print(new_data)

# Display the dictionary of incorrect spellings against correct spellings
print("Dictionary of incorrect spellings against correct spellings:")
print(incorrect_correct_dict)


In [ ]:
new_data[['ID','comment_without_stopword','misspellings','CorrectComment_Spell','total_incorrect_words']]

In [ ]:
new_data[new_data["total_incorrect_words"]>0]["misspellings"]

In [ ]:
# Filter rows where total_incorrect_words > 0
filtered_data = new_data[new_data['total_incorrect_words'] > 0]
filtered_data

In [ ]:
data.columns

### <font color="blue"> STORE COMMENT OF WORDLENGTH 3 AND WORDLENGTH 4 IN DIFFERENT COLUMN</font>

We cannot remove word which length is <3. Now We are going to do this. We try to remove wordlength < 3 and <4 and store in other columns

- We store comment of word length 3 and 4 in different column.
- So We can check what is the effect if we remove word of 3 character long and 4 character long against all length word.
- It also say effect of word in our vocabulary. 
- We also count total words in comment so we can bifergate comment in basic vs detail explaination

## CREATE COLUMN CleanComment3 , CleanComment4, TotalWordgte3, TotalWordgt4

In [ ]:
# Function to filter comments based on word length
def filter_comments(comment, word_length):
    return ' '.join([word for word in comment.split() if len(word) >= word_length])

# Function to count total words in comments based 
def count_words(comment):
    return len(comment.split())



# Word lengths to consider
word_lengths = [3, 4]  # Adjust as needed

# Create new columns for each word length
for length in word_lengths:
    column_name = f'CleanComment{length}'
    data[column_name] = data['CorrectComment_Spell'].apply(lambda x: filter_comments(x, length))
    
for length in word_lengths:
    word_length=f'TotalWordgte{length}'
    column_name = f'CleanComment{length}'
    data[word_length] = data[column_name].apply(lambda x: count_words(x))
   


In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score

# # Assuming you have a DataFrame called 'data' with columns 'Comment' and 'Comment_Length'
# # 'Comment' contains the text of the comments, and 'Comment_Length' contains the length of each comment
# # 'Type' contains the labels indicating whether the comment is "detail" (1) or "basic" (0)

# # Step 1: Split the dataset into features (X) and target variable (y)
# X = data['Comment_Length'].values.reshape(-1, 1)  # Reshape to fit the model input format
# y = data['Type']

# # Step 2: Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Step 3: Choose and train a machine learning model
# model = LogisticRegression()
# model.fit(X_train, y_train)

# # Step 4: Evaluate the model's performance
# y_pred = model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)

# Step 5: Use the trained model for prediction (optional)
# new_comment_length = 60  # Example length of a new comment
# predicted_type = model.predict([[new_comment_length]])
# print("Predicted Type:", predicted_type)


### <font color="blue">Differentiate "Basic" and "Detailed" Comment</blue>

In [ ]:
def comment_type(x):
    if len(x) > 50 :
        return "Detail"
    else:
        return "Basic"
    
# Word lengths to consider
word_lengths = [3, 4]  # Adjust as needed

for length in word_lengths:
    comment=f'ComemntType{length}'
    column_name = f'CleanComment{length}'
    data[f'{column_name}_Length']=data[column_name].apply(lambda x: len(x))
    data[comment] = data[column_name].apply(lambda x: comment_type(x))
    


#### CleanComment3 : Contain comment after removing word legnth < 3


#### TotalWordge3: Contain total word in CleanComment3. Useful to find comment is basic or detail.

#### CleanComment4: Contain comment after removing word length < 4

#### TotalWordge4: Contain total word in CleanComment4. Useful to find comment is basic or detail.

In [ ]:
data.rename(columns = {'ComemntType3':'CommentType3','ComemntType4':'CommentType4'}, inplace = True) 

- CleanComment column with number contain the comment after removing stopwords.

In [ ]:
data["CommentType3"].value_counts()

### <font color="blue">Perform Wordfrequency chart against Word Length. </font>
#### So it show importance of word and why we need to remove word whose length is < 3 character long

In [ ]:
# Function to count word frequency for a given column
def count_word_frequency(column):
    word_length_counts = column.str.split().explode().str.len().value_counts().sort_index()
    return word_length_counts
# Plot word frequency against word length for each column in one row
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharey=True)

# Column names
column_names = ['CleanComment', 'CleanComment3', 'CleanComment4']
    
for i, column_name in enumerate(column_names):
        word_length_counts = count_word_frequency(data[column_name])
        axes[i].bar(word_length_counts.index, word_length_counts.values, color='skyblue')
        axes[i].set_title(column_name)
        axes[i].set_xlabel('Word Length')
        axes[i].set_ylabel('Frequency')

plt.suptitle('Word Frequency vs. Word Length', y=1.05)
plt.tight_layout()
plt.show()


#### Count total number of word in each type of comment. comment without removing word and comment with wordlen > 3 and >4

In [ ]:
# Create a dictionary to store the word counts for each category
word_counts = {}

# Calculate word counts for each category for each column
for column_name in column_names:
    print(column_name)
#     comment_length=data[column_name].str.len()
    words = data[column_name].str.split().explode()
    total_words = len(words)
    words_lt_3 = words[words.str.len() < 3].count()
    words_eq_3 = words[words.str.len()==3].count()
    words_gte_4 = words[words.str.len() >= 4].count()
    
    word_counts[column_name] = {
        'Total Words': total_words,
        'Words < 3': words_lt_3,
        'Words == 3': words_eq_3,
        'Words >= 4': words_gte_4
    }

# Create a DataFrame from the word counts dictionary
word_counts_df = pd.DataFrame(word_counts)

# Display the word counts DataFrame
print(word_counts_df)

In [ ]:
word_counts_df

In [ ]:
statistics = {}

# Calculate statistics for each column
for column_name in column_names:
    word_lengths = data[column_name].str.split().explode().str.len()
    statistics[column_name] = {
        'Mean': word_lengths.mean(),
        'Median': word_lengths.median(),
        'Standard Deviation': word_lengths.std(),
        'Minimum': word_lengths.min(),
        'Maximum': word_lengths.max()
    }

# Create a DataFrame from the statistics dictionary
statistics_df = pd.DataFrame(statistics)

# Display the statistics DataFrame
print(statistics_df)

### <font color="blue"> CHECK 3 AND 4 WORD LENGTH WORDS FREQUNECY</blue>

In [ ]:
tokenized_comments = [word_tokenize(comment) for comment in data["CorrectComment_Spell"]]

# Flatten the list of tokenized comments
flat_tokens = [token for sublist in tokenized_comments for token in sublist if len(token)==3]

# Create a frequency distribution
freq_dist = FreqDist(flat_tokens)

# Display the most common words and their frequencies
print("Top 20 Most Common Words:")
print(freq_dist.most_common())

## <FONT COLOR="BLUE">LEMMATIZATION USING NLTK</FONT>

In [ ]:
data["cleancomment2"]=""
data["cleancomment3"]=""
data["cleancomment4"]=""
bows3=[]
bows4=[]
data['bow']=""

- cleancomment2,cleancomment3,cleancomment4 contain comment after the apply lemmatization.
- It only contain word which match with POS ['VERB', 'NOUN', 'ADJ', 'ADV']

In [ ]:
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS

# Drop rows with missing values in 'comment_without_stopword' column
data = data.dropna(subset=['CorrectComment_Spell'])

# Convert any float values in 'comment_without_stopword' column to empty string
data['CorrectComment_Spell'] = data['CorrectComment_Spell'].apply(lambda x: '' if isinstance(x, float) else x)

# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

# Apply bigram and trigram
bow = Phrases(data['CorrectComment_Spell'], min_count=1, threshold=1, connector_words=ENGLISH_CONNECTOR_WORDS)
data['bow'] = data['CorrectComment_Spell'].apply(lambda x: list(bow[x.split()]))

# Function to perform lemmatization with POS tagging and create new columns
def lemmatize_with_pos_and_create_columns(text):
    doc = nlp(text)
    lemmatized_words = [token.lemma_ for token in doc if token.pos_ in ['VERB', 'NOUN', 'ADJ', 'ADV']]
    cleancomment2 = " ".join(lemmatized_words)
    cleancomment3 = " ".join([word for word in lemmatized_words if len(word) > 3])
    cleancomment4 = " ".join([word for word in lemmatized_words if len(word) > 4])
    return cleancomment2, cleancomment3, cleancomment4

# Apply the function to each row in the dataframe
data[['cleancomment2', 'cleancomment3', 'cleancomment4']] = data["bow"].apply(lambda x: pd.Series(lemmatize_with_pos_and_create_columns(' '.join(x))))

# # Print the updated dataframe
# print(data.head())


In [ ]:
data["cleancomment3"][0]


In [ ]:
import pandas as pd
from spellchecker import SpellChecker

# Initialize SpellChecker with English language
spell = SpellChecker(language='en')

# Define words to exclude from correction
excluded_words = {'scanf', 'printf', 'getch', 'stdio', 'conio', 'clrscr', 'getchar',  
                  'clrscr', 'void', 'int', 'datatype','runtime','specifier'}

# Define manual corrections for specific words
manual_corrections = {'flot': 'float', 'lot':'float','clrsccr': 'clrscr','flost':'float','\x91clrscr':'clrscr',
                      'specifi':'specifier','specific':'specifier'}

# Function to spellcheck each comment
def spellcheck_comment(comment):
    if comment is None or comment == "":
        return "", {}, 0

    # Tokenize the comment
    tokens = comment.split()

    # Initialize dictionary to store misspelled words and their corrections
    misspellings = {}

    # Iterate through each word in the comment
    for word in tokens:
        # Check if the word is in the excluded list
        if word.lower() in excluded_words:
            continue  # Skip correction for excluded words
        
        # Check if the word is misspelled
        corrected_word = spell.correction(word)
        
        # Check if the corrected word is one of the manual corrections
        if corrected_word in manual_corrections:
            corrected_word = manual_corrections[corrected_word]
        
        if corrected_word != word:
            # Store the misspelled word and its corrected form in the dictionary
            misspellings[word] = corrected_word

    # Replace misspelled words with correct spelling in the comment
    corrected_tokens = []
    total_incorrect_words = 0
    for word in tokens:
        corrected_word = misspellings.get(word)
        if corrected_word is not None:
            corrected_tokens.append(corrected_word)
            total_incorrect_words += 1
        else:
            corrected_tokens.append(word)
    
    corrected_comment = ' '.join(corrected_tokens)

    return corrected_comment, misspellings, total_incorrect_words

# Apply spellchecking to each comment in the dataframe for cleancomment2, cleancomment3, and cleancomment4
data['CorrectComment2_Spell'], data['misspellings2'], data['total_incorrect_words2'] = zip(*data['cleancomment2'].apply(spellcheck_comment))
data['CorrectComment3_Spell'], data['misspellings3'], data['total_incorrect_words3'] = zip(*data['cleancomment3'].apply(spellcheck_comment))
data['CorrectComment4_Spell'], data['misspellings4'], data['total_incorrect_words4'] = zip(*data['cleancomment4'].apply(spellcheck_comment))

# Create a dictionary of incorrect spellings against correct spellings for cleancomment2
incorrect_correct_dict2 = {}
for misspelling_dict in data['misspellings2']:
    for misspelled_word, corrected_word in misspelling_dict.items():
        if misspelled_word not in incorrect_correct_dict2:
            incorrect_correct_dict2[misspelled_word] = corrected_word

# Create a dictionary of incorrect spellings against correct spellings for cleancomment3
incorrect_correct_dict3 = {}
for misspelling_dict in data['misspellings3']:
    for misspelled_word, corrected_word in misspelling_dict.items():
        if misspelled_word not in incorrect_correct_dict3:
            incorrect_correct_dict3[misspelled_word] = corrected_word

# Create a dictionary of incorrect spellings against correct spellings for cleancomment4
incorrect_correct_dict4 = {}
for misspelling_dict in data['misspellings4']:
    for misspelled_word, corrected_word in misspelling_dict.items():
        if misspelled_word not in incorrect_correct_dict4:
            incorrect_correct_dict4[misspelled_word] = corrected_word

# Create new dataframes with required columns
new_data2 = data[['ID', 'Name', 'Comment1', 'cleancomment2', 'misspellings2', 'CorrectComment2_Spell', 'total_incorrect_words2']]
new_data3 = data[['ID', 'Name', 'Comment1', 'cleancomment3', 'misspellings3', 'CorrectComment3_Spell', 'total_incorrect_words3']]
new_data4 = data[['ID', 'Name', 'Comment1', 'cleancomment4', 'misspellings4', 'CorrectComment4_Spell', 'total_incorrect_words4']]

# Display the corrected comments and misspelled words for cleancomment2
print(new_data2)

# Display the corrected comments and misspelled words for cleancomment3
print(new_data3)

# Display the corrected comments and misspelled words for cleancomment4
print(new_data4)


In [ ]:

%%capture cap

def print_pos_tag(sentence):
    doc = nlp(sentence)
    for token in doc:
         if token.pos_ in ["VERB", "ADJ", "ADV", "NOUN"]:
            print(token.text, token.pos_)

# Apply the function to each row in the "cleancomment3" column
for sentence in data["cleancomment3"]:
#       for w in sentence:
         print_pos_tag(sentence)
          #print(sentence)

# spellcheck2(data,'comment_without_stopword')
# spellcheck2(data,'CleanComment3')
# spellcheck2(data,'CleanComment4')

with open('c:\\Research\\POS1.txt', 'w',encoding="utf-8") as file:
    file.write(cap.stdout)

In [ ]:
def print_pos_tag(sentence):
    doc = nlp(sentence)
    for token in doc:
         if token.pos_ in ["VERB", "ADJ", "ADV", "NOUN"]:
            print(token.text, token.pos_)

print_pos_tag(lem_teacher_answer)

In [ ]:
# tokenized_comments = [word_tokenize(comment) for comment in data["cleancomment3"]]

# # Flatten the list of tokenized comments
# flat_tokens = [token for sublist in tokenized_comments for token in sublist if len(token)>3]

# # Create a frequency distribution
# freq_dist = FreqDist(flat_tokens)

# # Display the most common words and their frequencies
# print("Top 20 Most Common Words:")
# print(freq_dist.most_common())

In [ ]:
newdata=data.copy()

In [ ]:
# Find indices of rows with empty strings or lists in the "cleancomment2" column
empty_indices = newdata[newdata['cleancomment3'].apply(lambda x: len(x) == 0)].index

# Print the indices of rows with empty strings or lists
print("Indices of rows with empty strings or lists:",(empty_indices))
print("Length of newdata dataframe is:",len(newdata))
print("Total row is :",len(newdata)-len(empty_indices))

In [ ]:
# Remove rows with empty lists in the "cleancomment2" column
newdata = newdata[newdata['cleancomment3'].apply(lambda x: len(x) > 0)]

# Reset index
newdata.reset_index(drop=True, inplace=True)

# Display the updated DataFrame
print(len(newdata))

### <FONT COLOR="BLUE">ANALYSE WORD FREQUENCY BEFORE AND AFTER LEMMATIZATION</FONT>

In [ ]:
data["lem"]=""
freq_dist=" "

In [ ]:
tokenized_comments = [word_tokenize(comment) for comment in newdata["CorrectComment3_Spell"]]

# Flatten the list of tokenized comments
flat_tokens = [token for sublist in tokenized_comments for token in sublist ] #if len(token)>4]

# Create a frequency distribution
freq_dist = FreqDist(flat_tokens)

# Display the most common words and their frequencies
print("Top 20 Most Common Words:")
print(freq_dist.most_common(20))
# data["lem3"] = data["cleancomment3"].apply(lambda x: "".join(x))

### <FONT COLOR="BLUE">ANALYSE WORD FREQUENCY BEFORE AND AFTER LEMMATIZATION</FONT>

### Find Frequncy Distribution After lemmmatization cleancomment2 (Contain all length word) 

In [ ]:
data["lem"]=""
freq_dist=" "

In [ ]:
tokenized_comments = [word_tokenize(comment) for comment in newdata["CorrectComment2_Spell"]]

# Flatten the list of tokenized comments
flat_tokens = [token for sublist in tokenized_comments for token in sublist ] #if len(token)>4]

# Create a frequency distribution
freq_dist = FreqDist(flat_tokens)

# Display the most common words and their frequencies
print("Top 20 Most Common Words:")
print(freq_dist.most_common(20))
# data["lem3"] = data["cleancomment3"].apply(lambda x: "".join(x))

### <FONT COLOR="BLUE"> Word Frequency Distribution Plot After Lemmatization </FONT>

In [ ]:
# from nltk.tokenize import word_tokenize
# from nltk.probability import FreqDist
# from matplotlib import pyplot as plt

def plot_top_words(ax, column, title):
    # Tokenize comments in the specified column
    tokenized_comments = [word_tokenize(comment) for comment in column]
    
    # Flatten the list of tokenized comments
    flat_tokens = [token for sublist in tokenized_comments for token in sublist]
    
    # Create a frequency distribution
    freq_dist = FreqDist(flat_tokens)
    
    # Get the top 20 most common words
    top_words = freq_dist.most_common(20)
    x, y = zip(*top_words)
    
    # Plot the top 20 most common words
    ax.bar(x, y)
    ax.set_title(title)
    ax.set_xlabel('Words')
    ax.set_ylabel('Frequency')
    ax.tick_params(axis='x', rotation=45)

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Plot for cleancomment2
# plot_top_words(axes[0], newdata["cleancomment2"], 'Top 20 Most Common Words in cleancomment2')

# Plot for cleancomment3
plot_top_words(axes[0], newdata["CorrectComment3_Spell"], 'Top 20 Most Common Words in cleancomment3')

# Plot for cleancomment4
plot_top_words(axes[1], newdata["CorrectComment4_Spell"], 'Top 20 Most Common Words in cleancomment4')

plt.tight_layout()
plt.show()



### WORDFREQUENCY PLOT AFTER LEMMATIZATION

In [ ]:
# Function to count word frequency for a given column
def count_word_frequency(column):
    word_length_counts = column.str.split().explode().str.len().value_counts().sort_index()
    return word_length_counts
# Plot word frequency against word length for each column in one row
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharey=True)

# Column names
column_names = ['CleanComment', 'CleanComment3', 'CleanComment4']
    
for i, column_name in enumerate(column_names):
        word_length_counts = count_word_frequency(newdata[column_name])
        axes[i].bar(word_length_counts.index, word_length_counts.values, color='skyblue')
        axes[i].set_title(column_name)
        axes[i].set_xlabel('Word Length')
        axes[i].set_ylabel('Frequency')

plt.suptitle('Word Frequency vs. Word Length Before Lemmatization', y=1.05)
plt.tight_layout()
plt.show()
# Plot word frequency against word length for each column in one row
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharey=True)

# Column names
column_names = ['CorrectComment2_Spell', 'CorrectComment3_Spell', 'CorrectComment4_Spell']
    
for i, column_name in enumerate(column_names):
        word_length_counts = count_word_frequency(newdata[column_name])
        axes[i].bar(word_length_counts.index, word_length_counts.values, color='skyblue')
        axes[i].set_title(column_name)
        axes[i].set_xlabel('Word Length')
        axes[i].set_ylabel('Frequency')

plt.suptitle('Word Frequency vs. Word Length After Lemmatization', y=1.05)
plt.tight_layout()
plt.show()


### Find Statistic of Before and After Lemmatization of All Word, Word Len > 3 and Word Len > 4

In [ ]:
# Create a dictionary to store the word counts for each category
word_counts = {}

# Calculate word counts for each category for each column
for column_name in column_names:
    words = newdata[column_name].str.split().explode()
    total_words = len(words)
    words_lt_3 = words[words.str.len() < 3].count()
    words_eq_3 = words[words.str.len() == 3].count()
    words_gte_4 = words[words.str.len() >= 4].count()
    
    word_counts[column_name] = {
        'Total Words': total_words,
        'Words < 3': words_lt_3,
        'Words = 3': words_eq_3,
        'Words >= 4': words_gte_4
    }

# Create a DataFrame from the word counts dictionary
newword_counts_df = pd.DataFrame(word_counts)

# Display the word counts DataFrame
print("\t\t\tWord Count Statistics After Lemmatization\n=====================================================================================")
print(newword_counts_df)
print("\n\n\t\t\tWord Count Statistics before Lemmatization\n=====================================================================================")
print(word_counts_df)

### <FONT COLOR="BLUE"> PRINT WORD CLOUD </FONT>

In [ ]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
def wordfreq(data,colname):
    
    tokenized_comments = [word_tokenize(comment) for comment in newdata[colname]]

    # Flatten the list of tokenized comments
    flat_tokens = [token for sublist in tokenized_comments for token in sublist ] #if len(token)>4]

    # Create a frequency distribution
    freq_dist = FreqDist(flat_tokens)
    return freq_dist

In [ ]:
# from wordcloud import WordCloud
# import matplotlib.pyplot as plt

col=['CorrectComment2_Spell','CorrectComment3_Spell','CorrectComment4_Spell']
# freqdist(newdata,l)
# Generate word clouds
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, (column, ax) in enumerate(zip(['CorrectComment2_Spell', 'CorrectComment3_Spell', 'CorrectComment4_Spell'], axes)):
#     print(f"{i}--->{column}--->{ax}")
    wordcloud = WordCloud(width=300, height=300, background_color='white', max_font_size=100, colormap='viridis').generate_from_frequencies(wordfreq(newdata,column))
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.set_title(f'Word Cloud - {column}')
    ax.axis('off')

plt.tight_layout()
plt.show()

### <FONT COLOR="BLUE"> COMMENT CATEGORY </FONT>

In [ ]:
# Define error and solution keywords
error_keywords = ["error", "syntax", "logical", "runtime", "exception", "bug", "issue", "mistake", "flaw", "incorrect", "fault", "problem", "debug", "crash"]
solution_keywords = ["solution", "fix", "resolve", "correct", "handle", "patch", "repair", "address", "amend", "rectify", "debugging", "workaround"]
# Function to categorize comments into error only, solution only, or both
def categorize_comment(comment):
    contains_error = any(keyword in comment.lower() for keyword in error_keywords)
    contains_solution = any(keyword in comment.lower() for keyword in solution_keywords)
    
    if contains_error and not contains_solution:
        return "Error Only"
    elif contains_solution and not contains_error:
        return "Solution Only"
    elif contains_error and contains_solution:
        return "Both Error and Solution"
    else:
        return "None"

# Apply the categorization function to each comment in the DataFrame column "cleancomment2"
newdata['Comment_Category'] = newdata['CorrectComment3_Spell'].apply(categorize_comment)

# Display the DataFrame with the new column
print(newdata[['ID','CorrectComment3_Spell', 'Comment_Category']])

### <FONT COLOR="BLUE">FIND MATCHING ANSWER KEYWORD</BLUE>

In [ ]:
# Function to find matching keywords between student's comment and teacher answer keywords
d=0
def find_matching_keywords(comment, keywords):
#     matching_keywords = [keyword for keyword in keywords if keyword in comment.split()]
    matching_keywords=[]
    for keyword in keywords:
        if keyword in comment.split():
            matching_keywords.append(keyword)
    global d
    d=d+1
    if matching_keywords:
        print(f"{d}-->{comment}-->{','.join(matching_keywords)}")
        return ','.join(matching_keywords)
    else:
        return "No Matching"

# Apply the function to each row in the dataframe

teacher_keywords = ['variable', 'declaration', 'integer', 'int','undeclare', 'declare','datatype']
newdata['Match_with_Answer_keywords3'] = newdata['CorrectComment3_Spell'].apply(lambda x: find_matching_keywords(x, teacher_keywords))

# Display the updated DataFrame
print(newdata)

In [ ]:
# Function to find matching keywords between student's comment and teacher answer keywords
d=0
def find_matching_keywords(comment, keywords):
#     matching_keywords = [keyword for keyword in keywords if keyword in comment.split()]
    matching_keywords=[]
    for keyword in keywords:
        if keyword in comment.split():
            matching_keywords.append(keyword)
    global d
    d=d+1
    if matching_keywords:
        print(f"{d}-->{comment}-->{','.join(matching_keywords)}")
        return ','.join(matching_keywords)
    else:
        return "No Matching"

# Apply the function to each row in the dataframe

teacher_keywords = teacher_keywords = ['variable', 'declaration', 'integer', 'undeclare', 'declare','datatype']
newdata['Match_with_Answer_keywords2'] = newdata['CorrectComment2_Spell'].apply(lambda x: find_matching_keywords(x, teacher_keywords))

# Display the updated DataFrame
print(newdata)

### <FONT COLOR="BLUE"> STORE ERROR AND SOLUTION MATCHING KEYWORD </FONT>

In [ ]:
import pandas as pd

# Sample student comments
student_comments = newdata["CorrectComment2_Spell"]

# Define error and solution keywords
error_keywords = ["error", "wrong","syntax", "logical", "runtime", "exception", "bug", "issue", "mistake", "flaw", "incorrect", "fault", "problem", "debug", "crash"]
solution_keywords = ["solution", "fix", "resolve", "correct", "corrected","handle", "patch", "repair", "address", "amend", "rectify", "debugging", "workaround"]

# Initialize lists to store comments categorized as error only, solution only, and both error and solution
error_only_comments = []
solution_only_comments = []
both_error_solution_comments = []

# Initialize lists to store matching keywords for each category
error_keywords_matched = []
solution_keywords_matched = []
both_keywords_matched = []

# Iterate through each student comment
for comment in student_comments:
    contains_error = any(keyword in comment.lower() for keyword in error_keywords)
    contains_solution = any(keyword in comment.lower() for keyword in solution_keywords)
    
    error_matched_keywords = [keyword for keyword in error_keywords if keyword in comment.lower()]
    solution_matched_keywords = [keyword for keyword in solution_keywords if keyword in comment.lower()]
    
    if contains_error and not contains_solution:
        error_only_comments.append(comment)
        error_keywords_matched.append(','.join(error_matched_keywords))
        solution_keywords_matched.append('')
        both_keywords_matched.append('')
    elif contains_solution and not contains_error:
        solution_only_comments.append(comment)
        solution_keywords_matched.append(','.join(solution_matched_keywords))
        error_keywords_matched.append('')
        both_keywords_matched.append('')
    elif contains_error and contains_solution:
        both_error_solution_comments.append(comment)
        both_keywords_matched.append(','.join(error_matched_keywords + solution_matched_keywords))
        error_keywords_matched.append(','.join(error_matched_keywords))
        solution_keywords_matched.append(','.join(solution_matched_keywords))
    else:
        error_keywords_matched.append('')
        solution_keywords_matched.append('')
        both_keywords_matched.append('')
        
print(len(both_keywords_matched))
print(len(solution_keywords_matched))
print(len(error_keywords_matched))
# Add the matched keyword columns to the DataFrame
newdata['Error_Matched_Keywords2'] = error_keywords_matched
newdata['Solution_Matched_Keywords2'] = solution_keywords_matched
newdata['Both_Matched_Keywords2'] = both_keywords_matched

# Display the updated DataFrame
print(data)

In [ ]:
import pandas as pd

# Sample student comments
student_comments = newdata["CorrectComment3_Spell"]

# Define error and solution keywords
error_keywords = ["error", "wrong","syntax", "logical", "runtime", "exception", "bug", "issue", "mistake", "flaw", "incorrect", "fault", "problem", "debug", "crash"]
solution_keywords = ["solution", "fix", "resolve", "correct", "corrected","handle", "patch", "repair", "address", "amend", "rectify", "debugging", "workaround"]

# Initialize lists to store comments categorized as error only, solution only, and both error and solution
error_only_comments = []
solution_only_comments = []
both_error_solution_comments = []

# Initialize lists to store matching keywords for each category
error_keywords_matched = []
solution_keywords_matched = []
both_keywords_matched = []

# Iterate through each student comment
for comment in student_comments:
    contains_error = any(keyword in comment.lower() for keyword in error_keywords)
    contains_solution = any(keyword in comment.lower() for keyword in solution_keywords)
    
    error_matched_keywords = [keyword for keyword in error_keywords if keyword in comment.lower()]
    solution_matched_keywords = [keyword for keyword in solution_keywords if keyword in comment.lower()]
    
    if contains_error and not contains_solution:
        error_only_comments.append(comment)
        error_keywords_matched.append(','.join(error_matched_keywords))
        solution_keywords_matched.append('')
        both_keywords_matched.append('')
    elif contains_solution and not contains_error:
        solution_only_comments.append(comment)
        solution_keywords_matched.append(','.join(solution_matched_keywords))
        error_keywords_matched.append('')
        both_keywords_matched.append('')
    elif contains_error and contains_solution:
        both_error_solution_comments.append(comment)
        both_keywords_matched.append(','.join(error_matched_keywords + solution_matched_keywords))
        error_keywords_matched.append(','.join(error_matched_keywords))
        solution_keywords_matched.append(','.join(solution_matched_keywords))
    else:
        error_keywords_matched.append('')
        solution_keywords_matched.append('')
        both_keywords_matched.append('')
        
print(len(both_keywords_matched))
print(len(solution_keywords_matched))
print(len(error_keywords_matched))
# Add the matched keyword columns to the DataFrame
newdata['Error_Matched_Keywords3'] = error_keywords_matched
newdata['Solution_Matched_Keywords3'] = solution_keywords_matched
newdata['Both_Matched_Keywords3'] = both_keywords_matched

# Display the updated DataFrame
print(data)

In [ ]:
newdata.info()

In [ ]:
newdata[newdata["Comment_Category"]=="Both Error and Solution"]

### <FONT COLOR="BLUE">GENDER-WISE COUNT FOR COMMENT CATEGORY MEANS COMMENT IS ERROR, SOLUTION OR BOTH.</FONT>

In [ ]:
newdata.groupby(['Gender'])["Comment_Category"].count()

### <FONT COLOR="BLUE">FIND COSSINE SIMILARITY FOR ALL COMMENT</BLUE>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Teacher's answer and keywords
# teacher_answer = "error related to variable declartion. Forget to declare datatype of variable during declaration. so it generate syntax error. Solution : we have to declare variable integer because in printf statement it use %d as format specifier."
# teacher_keywords = ['variable', 'declaration', 'integer', 'undeclare', 'declare','datatype']

# Sample student comments (assuming they are stored in a DataFrame)
student_comments = newdata["CorrectComment3_Spell"]

# Initialize CountVectorizer with teacher's keywords
vectorizer = CountVectorizer(vocabulary=teacher_keywords)

# Transform teacher's answer and student comments separately
teacher_vector = vectorizer.fit_transform([teacher_answer])
student_vector = vectorizer.transform(student_comments)

# Calculate cosine similarity between teacher's answer and student comments
similarity_scores = cosine_similarity(teacher_vector, student_vector)

# Add cosine similarity scores to the DataFrame
newdata['Cosine_Similarity_Sent'] = similarity_scores.flatten()  # Flatten to make it 1D

# Display the DataFrame with cosine similarity scores
print(newdata[['ID', 'cleancomment3', 'Cosine_Similarity_Sent']])


In [ ]:
pip install spicy

In [ ]:
from scipy.spatial.similarity import cosine_similarity

# Sample vectors (replace with your actual data)
vec1 = [1, 2, 3]
vec2 = [4, 5, 6]

similarity = cosine_similarity(vec1, vec2)
print(similarity)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Teacher's answer (already lemmatized)
teacher_answer = lem_teacher_answer

# Student comments from DataFrame
student_comments = newdata["CorrectComment3_Spell"].tolist()

# Preprocess student comments (optional): lowercase, remove punctuation, etc.
# ... (Add your preprocessing steps here if needed)

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Transform text into TF-IDF vectors
teacher_features = vectorizer.fit_transform([teacher_answer])
student_features = vectorizer.transform(student_comments)  # Transform all comments at once

# Calculate cosine similarity for each student comment
from scipy.spatial.similarity import cosine_similarity

similarities = cosine_similarity(teacher_features, student_features)[0]

# Loop through student comments and print similarity scores
for i, comment in enumerate(student_comments):
  print(f"Cosine Similarity between Teacher Answer and Comment {i+1}:", similarities[i])


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Teacher's keywords
teacher_keywords = ['variable', 'declaration', 'integer', 'undeclare', 'declare','datatype']

# Sample student comments (assuming they are stored in a DataFrame)
student_comments = newdata["CorrectComment3_Spell"]

# Initialize CountVectorizer with teacher's keywords
vectorizer = CountVectorizer(vocabulary=teacher_keywords)

# Transform student comments
student_vector = vectorizer.transform(student_comments)

# Calculate cosine similarity between teacher's keywords and student comments
teacher_vector = vectorizer.transform([' '.join(teacher_keywords)])  # Combine teacher's keywords into a single document
similarity_scores = cosine_similarity(teacher_vector, student_vector)

# Add cosine similarity scores to the DataFrame
newdata['Cosine_Similarity_Keywords'] = similarity_scores.flatten()  # Flatten to make it 1D

In [ ]:
newdata.info()

In [ ]:
 newdata[["ID","Cosine_Similarity_Sent","Cosine_Similarity_Keywords"]]

### <font color="Blue">EVALUATION AND ASSIGN MARKS</font>

In [ ]:
# newdata2["Total_Answer_Keyword"]=newdata2['Match_with_Answer_keywords'].apply(lambda x:len(x.split(",")))

# Apply the function to create the "Total_Answer_Keyword" column
newdata["Total_Answer_Keyword2"] = newdata['Match_with_Answer_keywords2'].apply(lambda x: 0 if x == "No Matching" else len(x.split(",")))

# Display the updated DataFrame with the new column
# print(newdata2)


In [ ]:
# newdata2["Total_Answer_Keyword"]=newdata2['Match_with_Answer_keywords'].apply(lambda x:len(x.split(",")))

# Apply the function to create the "Total_Answer_Keyword" column
newdata["Total_Answer_Keyword3"] = newdata['Match_with_Answer_keywords3'].apply(lambda x: 0 if x == "No Matching" else len(x.split(",")))

# Display the updated DataFrame with the new column
# print(newdata2)


In [ ]:
# Apply the function to create the "Cosine_Value" column
newdata["Cosine_Value_Keywords"] = newdata['Cosine_Similarity_Keywords'].apply(lambda x: "High" if x > 0.70 else "Low")

In [ ]:
import numpy as np

# Define the total marks
total_marks = 50

# Define the criteria and corresponding marks allocation
marks_criteria = {
    'Cosine_Value_Keywords': {'High': 15, 'Low': 10},
    'CommentType3': {'Detail': 10, 'Basic': 5},
    'Comment_Category': {'Both Error and Solution': 10, 'Solution Only': 5, 'Error Only': 5, 'None': 2},
    'Total_Answer_Keyword2': {'>4': 15, '=4': 15, '<4': 10, '=0': 1}
}
teacher_keywords = ['variable', 'declaration', 'integer', 'undeclare', 'declare','datatype']
lenanskeyword= len(teacher_keywords)

# Function to assign marks based on the criteria
def assign_marks(row):
    marks = 0
    assignmarks=[]
    for criterion, allocation in marks_criteria.items():
        value = row[criterion]
#         print(f"{value}-->{allocation}")
        
        if value in allocation:
            marks += allocation[value]
            assignmarks.append(allocation[value])
        elif value >= (lenanskeyword)*0.60:
#             print(f"{value}-->{(lenanskeyword)*0.60}")
            marks += 15
            assignmarks.append(value)
        elif value <= (lenanskeyword)*0.60:
#             print(f"{value}-->{(lenanskeyword)*0.60}")
            marks += 10
            assignmarks.append(value)
        elif value == 0:
#             print(f"{value}-->{(lenanskeyword)*0.60}")
            assignmarks.append(value)
            marks += 0
        else:
             marks += 0  # If criterion not found, assign 0 marks
#         print(assignmarks)
    return marks
# Apply the function to the DataFrame to calculate the marks for each comment
newdata['Mark_With_4Rule'] = newdata.apply(assign_marks, axis=1)

# Display the updated DataFrame with marks
print(newdata)

In [ ]:
newdata.columns

In [ ]:
teacher_keywords = ['variable', 'declaration', 'integer','int', 'undeclare', 'declare', 'datatype']
total_answer_keywrods=len(teacher_keywords)
newdata['Total_Answer_keywords_per']=newdata['Total_Answer_Keyword3'].apply(lambda x:((x*100)/total_answer_keywrods))

In [ ]:
newdata[['ID','Total_Answer_Keyword3','Total_Answer_keywords_per']]

### <FONT COLOR='BLUE'> WRITE DATA TO FILE TO ASSIGN MANUAL MARK </FONT>

In [ ]:
newdata.info()

In [ ]:
manual=pd.DataFrame()

In [ ]:
manual=newdata[['ID','Name','Gender', 'Email Address', 'Timestatmp',
       'Comment1', 'CleanComment', 'comment_without_stopword',
       'Comment_Length', 'comment_after_bitrigram', 'CorrectComment_Spell',
       'misspellings', 'total_incorrect_words', 'CleanComment3',
       'CleanComment4', 'TotalWordgte3', 'TotalWordgte4',
       'CleanComment3_Length', 'CommentType3','cleancomment3','Match_with_Answer_keywords3','CorrectComment3_Spell','CommentType3','Comment_Category','Total_Answer_keywords_per','Total_Answer_Keyword3','Cosine_Similarity_Sent','Cosine_Similarity_Keywords']].copy()

In [ ]:
# csv_file_path = 'G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\Formanualmarkass-1.csv'
csv_file_path = 'G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\Formanualmarkass-12.csv'
manual.to_csv(csv_file_path, index=False)


In [ ]:
# Load the dataset
file_path = "G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\Formanualmarkass-1.csv"
mdata = pd.read_csv(file_path)

In [ ]:
mdata.info()

In [ ]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# import joblib

# # Load the training data
# file_path = r'G:\GitHub Classroom Assistant\Topic Modelling Dataset\TMC\Formanualmarkass-1.csv'
# data = pd.read_csv(file_path)

# # Preprocess the data (if necessary)
# # This may include handling missing values, encoding categorical variables, and scaling numerical features

# encoder_3rule = OneHotEncoder(drop='first', sparse=False)
# X_encoded_3rule = encoder_3rule.fit_transform(newdata[['CommentType3', 'Comment_Category']].values)
# X_processed_3rule = np.concatenate([X_encoded_3rule, newdata[['Total_Answer_keywords_per']].values], axis=1)
# y_3rule = data['Manual_Mark']

# # Step 4: Split the data into training and testing sets for the 3-rule method
# X_train_3rule, X_test_3rule, y_train_3rule, y_test_3rule = train_test_split(X_processed_3rule, y_3rule, test_size=0.2, random_state=42)
# # Split the data into training and testing sets
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Choose a machine learning algorithm and create an instance of the model
# model = RandomForestRegressor()

# # Train the model on the training data
# model.fit(X_train_3rule, y_train_3rule)

# # Evaluate the performance of the trained model
# y_pred = model.predict(X_test_3rule)
# mse = mean_squared_error(y_test_3rule, y_pred)
# mae = mean_absolute_error(y_test_3rule, y_pred)
# r2 = r2_score(y_test_3rule, y_pred)

# print("Mean Squared Error:", mse)
# print("Mean Absolute Error:", mae)
# print("R^2 Score:", r2)

# # Optionally, save the trained model for future use
# joblib.dump(model, 'trained_model.pkl')

import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Load the dataset
file_path = "G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\Formanualmarkass-12.csv"
mdata = pd.read_csv(file_path)

# Replace 'None' with 0 in the comment_category column
mdata['Comment_Category'].replace('None', 0, inplace=True)

# Select features and target variable
X = mdata[['CommentType3', 'Comment_Category', 'Total_Answer_keywords_per']]
y = mdata['Manual_Mark']

# Convert categorical features to numerical using one-hot encoding
X_encoded = pd.get_dummies(X, drop_first=True)
# print(mdata.Comment_Category)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Train Random Forest model
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

# Predict marks for test set
y_pred = rf_model.predict(X_test)

# Calculate accuracy metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print accuracy metrics
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R^2 Score:", r2)


# Create a DataFrame to store predicted marks along with ID, manual marks, and difference
predictions_df = pd.DataFrame({'ID': X_test.index, 'Manual_Marks': y_test, 'Predicted_Marks': y_pred})
predictions_df['Difference'] = predictions_df['Manual_Marks'] - predictions_df['Predicted_Marks']

# Print the DataFrame
print(predictions_df[['ID', 'Manual_Marks', 'Predicted_Marks', 'Difference']])
print(len(predictions_df))

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, make_scorer
import numpy as np

# Load the dataset
file_path = "G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\Formanualmarkass-1.csv"
mdata = pd.read_csv(file_path)

# Define features and target variable
X = mdata[['CommentType3', 'Comment_Category', 'Total_Answer_keywords_per']]
y = mdata['Manual_Mark']

# Convert categorical features to numerical using one-hot encoding
X_encoded = pd.get_dummies(X)

# Define models
models = {
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "Support Vector Machines": SVR(),
    "Linear Regression": LinearRegression()
}

# Define scoring metric (mean squared error)
scorer = make_scorer(mean_squared_error)

# Evaluate models using cross-validation
results = {}
for name, model in models.items():
    cv_scores = cross_val_score(model, X_encoded, y, cv=5, scoring=scorer)
    results[name] = cv_scores.mean()

# Print results
print("Algorithm\tMean Squared Error")
print("---------------------------------")
for name, score in results.items():
    print(f"{name}\t{score}")

# Select the best algorithm
best_algorithm = min(results, key=results.get)
print(f"\nBest Algorithm: {best_algorithm}")

# Train the best algorithm on the full dataset and make predictions
best_model = models[best_algorithm]
best_model.fit(X_encoded, y)
predicted_marks = best_model.predict(X_encoded)

# Round the predicted marks to the nearest integer (floor value)
predicted_marks_rounded = np.floor(predicted_marks)

# Print predicted marks with ID and manual marks
print("\nPredicted Marks (Rounded) with ID and Manual Marks:")
predicted_df = pd.DataFrame({
    'ID': mdata['ID'],
    'Manual_Mark': mdata['Manual_Mark'],
    'Predicted_Mark': predicted_marks_rounded
})
print(predicted_df)



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, cohen_kappa_score

# Load the dataset
file_path = "G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\Formanualmarkass-1.csv"
mdata = pd.read_csv(file_path)

# Select features and target variable
X = mdata[['CommentType3', 'Comment_Category', 'Total_Answer_keywords_per']]
y = mdata['Manual_Mark']

# Convert categorical features to numerical using one-hot encoding
X_encoded = pd.get_dummies(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Define models
models = {
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "Support Vector Machine": SVC(),
    "Logistic Regression": LogisticRegression()
}

# Train and evaluate models
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    kappa = cohen_kappa_score(y_test, y_pred)
    results[name] = {"report": report, "kappa": kappa}

# Print results
print("Algorithm\tAccuracy\tF1-Score\tPrecision\tRecall\tCohen's Kappa")
print("------------------------------------------------------------------")
for name, result in results.items():
    report = result['report']
    accuracy = report['accuracy']
    f1_score = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    kappa = result['kappa']
    print(f"{name}\t{accuracy:.4f}\t\t{f1_score:.4f}\t\t{precision:.4f}\t\t{recall:.4f}\t\t{kappa:.4f}")

# Select the best model based on Cohen's Kappa
best_model_name = max(results, key=lambda x: results[x]['kappa'])
best_model = models[best_model_name]

# Predict marks with the best model
best_model.fit(X_train, y_train)
y_pred_best = best_model.predict(X_test)

# Print ID, predicted marks, and manual marks with the best model
print("\nPredicted Marks with the Best Model:")
print("ID\tPredicted_Marks\tManual_Marks")
print("------------------------------------")
for idx, (pred_mark, manual_mark) in enumerate(zip(y_pred_best, y_test)):
    print(f"{idx}\t{pred_mark}\t\t{manual_mark}")


In [ ]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
# from sklearn.svm import SVR
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error

# # Load the dataset
# file_path = "G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\Formanualmarkass-1.csv"
# mdata = pd.read_csv(file_path)

# # Select features and target variable
# X = mdata[['CommentType3', 'Comment_Category', 'Total_Answer_keywords_per']]
# y = mdata['Manual_Mark']

# # Convert categorical features to numerical using one-hot encoding
# X_encoded = pd.get_dummies(X)

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# # Initialize models
# models = {
#     'Random Forest': RandomForestRegressor(),
#     'Gradient Boosting': GradientBoostingRegressor(),
#     'Support Vector Machine': SVR(),
#     'Linear Regression': LinearRegression()
# }

# # Train each model and calculate MSE
# mse_results = {}
# for name, model in models.items():
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)
#     mse = mean_squared_error(y_test, y_pred)
#     mse_results[name] = mse

# # Print MSE values in tabular form
# print("Algorithm\t\tMean Squared Error")
# print("--------------------------------------")
# for name, mse in mse_results.items():
#     print(f"{name.ljust(20)}\t{mse:.4f}")

# # Find the best algorithm based on MSE
# best_algorithm = min(mse_results, key=mse_results.get)
# print(f"\nBest Algorithm: {best_algorithm}")

# # Print ID, manual mark, and predicted mark of the best algorithm
# best_model = models[best_algorithm]
# best_model.fit(X, y)  # Train on the full dataset
# y_pred_best = best_model.predict(X)  # Predict on the full dataset
# results = pd.DataFrame({'ID': data['ID'], 'Manual_Mark': data['Manual_Mark'], 'Predicted_Mark': y_pred_best})
# print("\nID\tManual Mark\tPredicted Mark")
# print(results)

import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Load the dataset
file_path = "G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\Formanualmarkass-1.csv"
mdata = pd.read_csv(file_path)

# Encode categorical variables
encoder = LabelEncoder()
mdata['CommentType3'] = encoder.fit_transform(mdata['CommentType3'])
mdata['Comment_Category'] = encoder.fit_transform(mdata['Comment_Category'])

# Select features and target variable
X = mdata[['CommentType3', 'Comment_Category', 'Total_Answer_keywords_per']]
y = mdata['Manual_Mark']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define models
models = {
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "Support Vector Machine": SVR(),
    "Linear Regression": LinearRegression()
}

# Train models and calculate MSE
mse_results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_results[name] = mse

# Print MSE for each model
print("Algorithm\tMean Squared Error")
print("--------------------------------------")
for name, mse in mse_results.items():
    print(f"{name}\t\t{mse}")

# Find best model
best_algorithm = min(mse_results, key=mse_results.get)
print(f"\nBest Algorithm: {best_algorithm}")

# Train the best model on the full dataset
best_model = models[best_algorithm]
best_model.fit(X, y)

# Predict marks for the full dataset
y_pred_full = best_model.predict(X)

# Print ID, manual mark, and predicted mark of the best algorithm
results = pd.DataFrame({'ID': mdata['ID'], 'Manual_Mark': mdata['Manual_Mark'], 'Predicted_Mark': y_pred_full})
print(results)


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, classification_report, cohen_kappa_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Load the dataset
# Load the dataset
file_path = "G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\Formanualmarkass-1.csv"
data = pd.read_csv(file_path)

# Encode categorical variables
encoder = LabelEncoder()
data['CommentType3'] = encoder.fit_transform(data['CommentType3'])
data['Comment_Category'] = encoder.fit_transform(data['Comment_Category'])

# Select features and target variable
X = data[['CommentType3', 'Comment_Category', 'Total_Answer_keywords_per']]
y = data['Manual_Mark']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define models
models = {
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "Support Vector Machine": SVR(),
    "Linear Regression": LinearRegression()
}

# Train models and calculate MSE
mse_results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_results[name] = mse

# # Evaluate models based on classification metrics
# report_results = {}
# for name, model in models.items():
#     y_pred = model.predict(X_test)
#     y_pred_floor = [int(pred) for pred in y_pred]
#     report = classification_report(y_test, y_pred_floor, output_dict=True)
#     kappa = cohen_kappa_score(y_test, y_pred_floor)
#     report_results[name] = {'MSE': mse_results[name], 'Kappa': kappa, 'Report': report}

# Print metrics for each model
print("Algorithm\tMSE\tKappa\tAccuracy\tPrecision\tRecall\tF1-Score")
print("---------------------------------------------------------------------")
for name, results in report_results.items():
    mse = results['MSE']
    kappa = results['Kappa']
    accuracy = results['Report']['accuracy']
    precision = results['Report']['weighted avg']['precision']
    recall = results['Report']['weighted avg']['recall']
    f1_score = results['Report']['weighted avg']['f1-score']
    print(f"{name}\t{mse:.4f}\t{kappa:.4f}\t{accuracy:.4f}\t{precision:.4f}\t{recall:.4f}\t{f1_score:.4f}")

# Find best model based on kappa score
best_algorithm = max(report_results, key=lambda x: report_results[x]['Kappa'])
print(f"\nBest Algorithm based on Kappa: {best_algorithm}")

# Train the best model on the full dataset
best_model = models[best_algorithm]
best_model.fit(X, y)

# Predict marks for the full dataset
y_pred_full = best_model.predict(X)

# Print ID, manual mark, and predicted mark of the best algorithm
results = pd.DataFrame({'ID': data['ID'], 'Manual_Mark': data['Manual_Mark'], 'Predicted_Mark': y_pred_full})
results['Predicted_Mark'] = results['Predicted_Mark'].apply(lambda x: int(x))  # Round to nearest integer
print(results)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error

# Load the dataset
file_path = "G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\Formanualmarkass-1.csv"
data = pd.read_csv(file_path)

# One-hot encode categorical variables
X_encoded = pd.get_dummies(X)

# Split the encoded dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Initialize models
models = {
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "Support Vector Machine": SVR(),
    "Linear Regression": LinearRegression()
}

# # Calculate MSE for each model
# mse_results = {}
# for name, model in models.items():
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)
#     mse = mean_squared_error(y_test, y_pred)
#     mse_results[name] = mse

# # Print MSE for each model
# print("Algorithm\t\tMean Squared Error")
# print("------------------------------------------")
# for name, mse in mse_results.items():
#     print(f"{name}\t\t{mse:.4f}")

# # Find the best algorithm (minimum MSE)
# best_algo = min(mse_results, key=mse_results.get)
# print("\nBest Algorithm:", best_algo)

# # Calculate R-squared
# r_squared = r2_score(y_test, y_pred_best)

# # Calculate Mean Absolute Error (MAE)
# mae = mean_absolute_error(y_test, y_pred_best)

# print("R-squared:", r_squared)
# print("Mean Absolute Error (MAE):", mae)

# Print ID, manual marks, and predicted marks for the best algorithm
# best_model = models[best_algo]
# best_model.fit(X_train, y_train)
# y_pred_best = best_model.predict(X_test)
# rounded_pred_marks = np.floor(y_pred_best)  # Round down to the nearest integer
# results = pd.DataFrame({'ID': data['ID'], 'Manual_Mark': data['Manual_Mark'], 'Predicted_Mark': rounded_pred_marks})
# print("\nID\tManual Mark\tPredicted Mark")
# print(results)
# Print ID, manual marks, and predicted marks for the best algorithm
# best_model = models[best_algo]
# best_model.fit(X_train, y_train)
# y_pred_best = best_model.predict(X_test)
# rounded_pred_marks = np.floor(y_pred_best) # Round down to the nearest integer
# results = pd.DataFrame({'ID': X_test.index, 'Manual_Mark': y_test, 'Predicted_Mark': rounded_pred_marks})
# print("\nID\tManual Mark\tPredicted Mark")
# print(results)



# Initialize dictionaries to store R-squared and MAE for each model
r_squared_results = {}
mae_results = {}

# Calculate R-squared and MAE for each model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r_squared = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse_results[name] = mse
    r_squared_results[name] = r_squared
    mae_results[name] = mae

# Print MSE, R-squared, and MAE for each model
print("Algorithm\t\tMean Squared Error\tR-squared\t\tMean Absolute Error")
print("---------------------------------------------------------------------------")
for name, mse in mse_results.items():
    print(f"{name}\t\t{mse:.4f}\t\t{r_squared_results[name]:.4f}\t\t{mae_results[name]:.4f}")

# Find the best algorithm based on MSE, R-squared, and MAE
best_algo_mse = min(mse_results, key=mse_results.get)
best_algo_r_squared = max(r_squared_results, key=r_squared_results.get)
best_algo_mae = min(mae_results, key=mae_results.get)

print("\nBest Algorithm based on MSE:", best_algo_mse)
print("Best Algorithm based on R-squared:", best_algo_r_squared)
print("Best Algorithm based on MAE:", best_algo_mae)



### As we find linear regression model is best. now we write code to train this model and save output.

### <font color="Blue"> TRAIN MODEL & SAVE (LINEAR REGRESSION) </font>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import joblib

# Load the dataset
file_path = "G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\Formanualmarkass-1.csv"
data = pd.read_csv(file_path)

# Extract features and target variable
X = data[['CommentType3', 'Comment_Category', 'Total_Answer_keywords_per']]
y = data['Manual_Mark']

# One-hot encode categorical variables
X_encoded = pd.get_dummies(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Initialize and train the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Save the trained model to a file
model_file_path = "G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\linear_regression_model.joblib"
joblib.dump(model, model_file_path)

print("Linear Regression model saved successfully.")


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
import joblib

# Load the dataset
file_path = "G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\Formanualmarkass-1.csv"
data = pd.read_csv(file_path)

# Extract features and target variable
X = data[['CommentType3', 'Comment_Category', 'Total_Answer_keywords_per']]
y = data['Manual_Mark']

# One-hot encode categorical variables
encoder = OneHotEncoder()
X_encoded = encoder.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Initialize and train the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Save the trained model and encoder to a file
model_file_path = "G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\linear_regression_model1.joblib"
joblib.dump((model, encoder), model_file_path)

print("Linear Regression model and encoder saved successfully.")


In [ ]:
from sklearn.metrics import accuracy_score

# Assuming your data is loaded into a pandas DataFrame called 'data' with columns:
# 'comment_id', 'ML_mark', 'rule_based_mark' (or any column containing human-assigned marks)

# Calculate absolute difference between marks
data['mark_difference'] = abs(data['ML_Mark'] - data['Manual_Mark'])

# Binarize the difference for accuracy calculation (correct if difference is 0)
data['correct'] = np.where(data['mark_difference'] == 0, 1, 0)  # 1 for correct prediction (0 difference)

# Calculate accuracy
accuracy = accuracy_score(data['correct'], np.ones(len(data)))  # Ones represent all expected correct predictions

print("Accuracy between Manual Marks and Training Model Predictions:", accuracy)


### <FONT COLOR="BLUE">MODEL EVALUATION USING MSE, MAE AND R2</FONT>

In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Load the dataset
file_path = "G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\Formanualmarkass-1.csv"
data = pd.read_csv(file_path)

# Extract actual manual marks and ML predicted marks
actual_marks = data['Manual_Mark']
predicted_marks = data['ML_Mark']

# Calculate evaluation metrics
mse = mean_squared_error(actual_marks, predicted_marks)
r2 = r2_score(actual_marks, predicted_marks)
mae = mean_absolute_error(actual_marks, predicted_marks)

# Print the evaluation metrics
print("Evaluation Metrics:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R-squared (R2): {r2:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")


In [ ]:
comment_data.columns

### <font color="Blue"> APPLY MODEL & SAVE (LINEAR REGRESSION) </font>

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import joblib

# Load the comment dataset
comment_file_path = "G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\Formanualmarkass-12.csv"
comment_data = pd.read_csv(comment_file_path)

# Assuming the comment dataset has the same features as the training dataset
# Extract features
X_comments = comment_data[['CommentType3', 'Comment_Category', 'Total_Answer_keywords_per']]

# Handle missing values
X_comments['Comment_Category'].fillna('No Match', inplace=True)  # Fill missing values with 'Unknown'

# Load the saved Linear Regression model and encoder
model_file_path = "G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\linear_regression_model1.joblib"
model, encoder = joblib.load(model_file_path)

# One-hot encode categorical variables using the loaded encoder
X_comments_encoded = encoder.transform(X_comments)

# Predict marks for the comment dataset
predicted_marks = model.predict(X_comments_encoded)

# Store predicted marks in a new column "ML_Mark"
comment_data['ML_Mark'] = np.ceil(predicted_marks)

# Print student ID, name, comment, and predicted marks
print("Student ID\tName\tComment\tPredicted Marks")
print("----------------------------------------------")
for index, row in comment_data.iterrows():
    print(f"{row['ID']}\t{row['Name']}\t{row['ML_Mark']}")

# Rewrite the CSV file with the "ML_Mark" column
comment_data.to_csv(comment_file_path, index=False)

print("CSV file rewritten with ML_Mark column.")


### <FONT COLOR='BLUE'>RULE BASED METHOD TO ASSIGN MARK (NEW FORMULA)</FONT>

In [ ]:
import pandas as pd

# Define teacher keywords (modify as needed)
teacher_keywords = ['variable', 'declaration', 'integer', 'undeclare', 'declare', 'datatype']

# Define the rule-based mapping for marks based on comment details (modify as needed)
def assign_marks(comment_type, comment_category, total_keywords, num_words):
  """
  Assigns marks based on the provided rules and comment details.

  Args:
      comment_type: The type of comment (e.g., Detail, Basic).
      comment_category: The category of the comment (e.g., Both Error & Solution).
      total_keywords: The total number of teacher keywords matched in the comment.
      num_words: The total number of words in the comment (optional for keyword percentage).

  Returns:
      An integer representing the assigned mark.
  """

  if comment_type == "Detail":
    if comment_category == "Both Error and Solution":
      if total_keywords >= 70:  # Check keyword percentage
        return 30
      elif total_keywords >= 50 and total_keywords < 70:
        return 25
      elif total_keywords >= 30 and total_keywords <50:
        return 20
      elif total_keywords >= 10 and total_keywords > 30:
        return 15
      else:
        return 0
    elif comment_category in ["Error Only", "Solution Only"]:
      if total_keywords >= 70:  # Check keyword percentage
        return 22
      elif total_keywords >= 50 and total_keywords < 70:
        return 18
      elif total_keywords >= 30 and total_keywords <50:
        return 14
      elif total_keywords >= 10 and total_keywords > 30:
        return 10
      else:
        return 0
    elif comment_category=='No Match':
      
      if total_keywords >= 70:  # Check keyword percentage
        return 20
      elif total_keywords >= 50 and total_keywords < 70:
        return 16
      elif total_keywords >= 30 and total_keywords <50:
        return 12
      elif total_keywords >= 10 and total_keywords > 30:
        return 8
      else:
        return 0
    
  elif comment_type == "Basic":
    if comment_category == "Both Error and Solution":
      if total_keywords >= 70:  # Check keyword percentage
        return 22
      elif total_keywords >= 50 and total_keywords < 70:
        return 18
      elif total_keywords >= 30 and total_keywords <50:
        return 14
      elif total_keywords >= 10 and total_keywords > 30:
        return 10
      else:
        return 0
    elif comment_category in ["Error Only", "Solution Only"]:
      if total_keywords >= 70:  # Check keyword percentage
        return 18
      elif total_keywords >= 50 and total_keywords < 70:
        return 14
      elif total_keywords >= 30 and total_keywords <50:
        return 10
      elif total_keywords >= 10 and total_keywords > 30:
        return 6
      else:
        return 0
    elif comment_category=='No Match':
      
      if total_keywords >= 70:  # Check keyword percentage
        return 15
      elif total_keywords >= 50 and total_keywords < 70:
        return 12
      elif total_keywords >= 30 and total_keywords <50:
        return 8
      elif total_keywords >= 10 and total_keywords >30:
        return 3
      else:
        return 0
    
# Load the comment dataset
comment_file_path = "G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\Formanualmarkass-12.csv"
comment_data = pd.read_csv(comment_file_path)

# Apply the rule-based assignment to the DataFrame
comment_data["Rule_Based_Mark"] = comment_data.apply(lambda row: assign_marks(row["CommentType3"], row["Comment_Category"], row['Total_Answer_keywords_per'], len(row["CorrectComment3_Spell"].split())), axis=1)
comment_data["Mark_Difference"] = comment_data.apply(lambda row: row["ML_Mark"]-row["Rule_Based_Mark"], axis=1)

print(comment_data["Rule_Based_Mark"])

comment_data.to_csv(comment_file_path, index=False)

print("CSV file rewritten with ML_Mark column.")

# Print a sample of the marked DataFrame (optional)
print(comment_data[["ID","Comment_Category","CorrectComment3_Spell", "Rule_Based_Mark","ML_Mark"]].head())


In [ ]:
comment_data [["ID","Name","ML_Mark","Rule_Based_Mark","Mark_Difference"]]

In [ ]:
comment_data["Mark_Difference"].describe()

### <FONT COLOR="BLUE">VISUALIZATION : HISTOGRAM FOR MARKS DIFFERENCE </FONT>

In [ ]:
plt.figure(figsize=(8, 5))
plt.hist(comment_data['Mark_Difference'], bins=10, edgecolor='black')
plt.xlabel('Difference between ML Mark and Rule-Based Mark')
plt.ylabel('Frequency')
plt.title('Histogram of Mark Difference')
plt.grid(axis='y')
plt.show()

# Create boxplot
plt.figure(figsize=(6, 5))
plt.boxplot(comment_data['Mark_Difference'])
plt.xlabel('Mark Difference')
plt.ylabel('Mark Difference')
plt.title('Boxplot of Mark Difference')
plt.grid(axis='y')
plt.show()

print("Summary Statistics of Mark Difference:")
print(comment_data['Mark_Difference'].describe())

In [ ]:
comment_data.info()

In [ ]:
comment_data.Mark_Difference.describe()

In [ ]:
# Calculate percentage difference for each observation
comment_data['Percentage_Difference'] = ((comment_data['ML_Mark'] - comment_data['Rule_Based_Mark']) / comment_data['Rule_Based_Mark']) * 100

# Compute the average percentage difference
average_percentage_difference = comment_data['Percentage_Difference'].mean()

print("Average Percentage Difference:", average_percentage_difference)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

# Define the data path
data_path = "G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\Formanualmarkass-12.csv"

# Read the data from the CSV file
comment_data = pd.read_csv(data_path)

# Option 1: Select only numerical features
numerical_features = ['Total_Answer_keywords_per', "ML_Mark", "Rule_Based_Mark"]
corr_matrix = comment_data[numerical_features].corr()

# Option 2: Convert appropriate features to numerical (if applicable)
# Assuming 'CommentType3' and 'Comment_Category' have categories that can be mapped to numbers
category_mapping = {'Basic': 1, 'Detail': 2, 'Excellent': 3}  # Define mapping for your categories
comment_data['CommentType3_Num'] = comment_data['CommentType3'].replace(category_mapping)
comment_data['Comment_Category_Num'] = comment_data['Comment_Category'].replace(category_mapping)

features = ['CommentType3_Num', 'Comment_Category_Num', 'Total_Answer_keywords_per', "ML_Mark", "Rule_Based_Mark"]
corr_matrix = comment_data[features].corr()

# Generate the heatmap
sns.heatmap(corr_matrix, annot=True)  # Set annot=True to display correlation values on the heatmap
plt.title("Correlation between Features")
plt.show()


### <font color="Blue"> ADJUST MARK BY APPLYING WEIGHT </font>

In [ ]:
weight = 0.5  # Adjust this value as needed

# Apply weighted adjustment to predicted marks
comment_data['Adjusted_Marks'] = comment_data.apply(lambda row: weighted_adjustment(row['ML_Mark'], row['Rule_Based_Mark'], weight), axis=1)

# Calculate average marks
comment_data['Avg_Mark'] = np.ceil((comment_data['ML_Mark'] + comment_data['Rule_Based_Mark']) / 2)

# Print the first few rows of the dataframe with the new columns
print(comment_data.head())

### <font color="blue"> VISUALIZATION AFTER ADJUSTMENT </font>

In [ ]:
plt.figure(figsize=(8, 5))
plt.hist(comment_data['Avg_Mark'], bins=10, edgecolor='black')
plt.xlabel('Difference between ML Mark and Rule-Based Mark')
plt.ylabel('Frequency')
plt.title('Histogram of Mark Difference')
plt.grid(axis='y')
plt.show()

# Create boxplot
plt.figure(figsize=(6, 5))
plt.boxplot(comment_data['Avg_Mark'])
plt.xlabel('Mark Difference')
plt.ylabel('Mark Difference')
plt.title('Boxplot of Mark Difference')
plt.grid(axis='y')
plt.show()

print("Summary Statistics of Mark Difference:")
print(comment_data['Avg_Mark'].describe())

### <font color="Blue"> FIND ACCURACY </font>

In [ ]:
#  Calculate metrics for ML method
ml_mse = mean_squared_error(comment_data['Actual_Manual_Marks'], comment_data['ML_Mark'])
ml_mae = mean_absolute_error(comment_data['Actual_Manual_Marks'], comment_data['ML_Mark'])
ml_r2 = r2_score(comment_data['Actual_Manual_Marks'], comment_data['ML_Mark'])

# Calculate metrics for rule-based method
rule_based_mse = mean_squared_error(comment_data['Actual_Manual_Marks'], comment_data['Rule_Based_Mark'])
rule_based_mae = mean_absolute_error(comment_data['Actual_Manual_Marks'], comment_data['Rule_Based_Mark'])
rule_based_r2 = r2_score(comment_data['Actual_Manual_Marks'], comment_data['Rule_Based_Mark'])

# Print metrics
print("Metrics for ML Method:")
print("MSE:", ml_mse)
print("MAE:", ml_mae)
print("R-squared:", ml_r2)
print()

print("Metrics for Rule-Based Method:")
print("MSE:", rule_based_mse)
print("MAE:", rule_based_mae)
print("R-squared:", rule_based_r2)

In [ ]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr

# Define the data path
data_path = "G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\Formanualmarkass-1.csv"

# Read the data from the CSV file
comment_data = pd.read_csv(data_path)

# Assuming the marks are in columns named 'ML_Mark', 'Rule_Based_Mark', and 'Actual_Mark'
# (modify these column names if they differ in your file)

# Calculate absolute difference for both methods
comment_data['ML_Diff'] = abs(comment_data['ML_Mark'] - comment_data['Manual_Mark'])
comment_data['Rule_Diff'] = abs(comment_data['Rule_Based_Mark'] - comment_data['Manual_Mark'])

# Accuracy for ML Model (within +/- 2 marks)
ml_accuracy = (comment_data['ML_Diff'] <= 2).sum() / len(comment_data)

# Accuracy for Rule-Based Method (within +/- 2 marks)
rule_accuracy = (comment_data['Rule_Diff'] <= 2).sum() / len(comment_data)

# Calculate agreement metrics
mae_ml = mean_absolute_error(comment_data['Manual_Mark'], comment_data['ML_Mark'])
mae_rule = mean_absolute_error(comment_data['Manual_Mark'], comment_data['Rule_Based_Mark'])

mse_ml = mean_squared_error(comment_data['Manual_Mark'], comment_data['ML_Mark'])
mse_rule = mean_squared_error(comment_data['Manual_Mark'], comment_data['Rule_Based_Mark'])

corr_ml, _ = pearsonr(comment_data['Manual_Mark'], comment_data['ML_Mark'])
corr_rule, _ = pearsonr(comment_data['Manual_Mark'], comment_data['Rule_Based_Mark'])

# Print the results
print("Accuracy:")
print(f"ML Model Accuracy: {ml_accuracy:.2f}")
print(f"Rule-Based Accuracy: {rule_accuracy:.2f}")

print("\nAgreement Metrics:")
print(f"MAE (ML): {mae_ml:.2f}")
print(f"MAE (Rule-Based): {mae_rule:.2f}")
print(f"MSE (ML): {mse_ml:.2f}")
print(f"MSE (Rule-Based): {mse_ml:.2f}")
print(f"Correlation (ML): {corr_ml:.2f}")
print(f"Correlation (Rule-Based): {corr_rule:.2f}")

# Compare metrics to determine better performing method

if ml_accuracy > rule_accuracy and (mae_ml < mae_rule or corr_ml > corr_rule):
  print("\nML Model performs better based on accuracy and agreement metrics.")
elif rule_accuracy > ml_accuracy and (mae_rule < mae_ml or corr_rule > corr_ml):
  print("\nRule-Based Method performs better based on accuracy and agreement metrics.")
else:
  print("\nThe performance is comparable. Consider further analysis or domain knowledge to decide.")


In [ ]:
import pandas as pd

# Define the data path
data_path = "G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\Formanualmarkass-12.csv"

# Read the data from the CSV file
comment_data = pd.read_csv(data_path)

# Assuming the marks are in columns named 'ML_Mark' and 'Rule_Based_Mark'
# (modify these column names if they differ in your file)

# Calculate the median for each method's predictions
ml_median = comment_data['ML_Mark'].median()
rule_based_median = comment_data['Rule_Based_Mark'].median()

# Calculate the absolute deviations from the median for each method
comment_data['ML_Abs_Dev'] = abs(comment_data['ML_Mark'] - ml_median)
comment_data['Rule_Based_Abs_Dev'] = abs(comment_data['Rule_Based_Mark'] - rule_based_median)

# Calculate the median absolute deviation (MAD) for each method
ml_mad = comment_data['ML_Abs_Dev'].median()
rule_based_mad = comment_data['Rule_Based_Abs_Dev'].median()

# Determine the method with lower MAD
if ml_mad < rule_based_mad:
    final_marks = comment_data['ML_Mark']
    print("ML Model predictions used for final marks due to lower MAD.")
else:
    final_marks = comment_data['Rule_Based_Mark']
    print("Rule-Based System predictions used for final marks due to lower MAD.")

# Print the final marks (assuming you want to see them)
print("\nFinal Marks:")
print(final_marks)


In [ ]:
import pandas as pd
from sklearn.metrics import mean_absolute_error

# Define the data path
data_path = "G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\Formanualmarkass-1.csv"

# Read the data from the CSV file
comment_data = pd.read_csv(data_path)

# Assuming the marks are in columns named 'ML_Mark', 'Rule_Based_Mark', and 'Actual_Mark'
# (modify these column names if they differ in your file)

# Calculate absolute differences for both methods
comment_data['ML_Diff'] = abs(comment_data['ML_Mark'] - comment_data['Manual_Mark'])
comment_data['Rule_Diff'] = abs(comment_data['Rule_Based_Mark'] - comment_data['Manual_Mark'])

# Cost per unit of overprediction and underprediction (adjust these values based on your context)
cost_overprediction_ml = 1  # Hypothetical cost for ML model overprediction
cost_underprediction_ml = 2  # Hypothetical cost for ML model underprediction

cost_overprediction_rule = 2  # Hypothetical cost for rule-based overprediction
cost_underprediction_rule = 1  # Hypothetical cost for rule-based underprediction

# Calculate cost of errors for both methods
comment_data['ML_Cost'] = comment_data['ML_Diff'] * (
    np.where(comment_data['ML_Diff'] > 0, cost_overprediction_ml, cost_underprediction_ml)
)
comment_data['Rule_Cost'] = comment_data['Rule_Diff'] * (
    np.where(comment_data['Rule_Diff'] > 0, cost_overprediction_rule, cost_underprediction_rule)
)

# Calculate total cost for each method
total_cost_ml = comment_data['ML_Cost'].sum()
total_cost_rule = comment_data['Rule_Cost'].sum()

# Determine the method with lower total cost
if total_cost_ml < total_cost_rule:
    final_marks = comment_data['ML_Mark']
    print(
        "ML Model predictions used for final marks due to lower total cost of errors."
    )
else:
    final_marks = comment_data['Rule_Based_Mark']
    print(
        "Rule-Based System predictions used for final marks due to lower total cost of errors."
    )

# Print the final marks (assuming you want to see them)
print("\nFinal Marks:")
print(final_marks)


In [ ]:
# import numpy as np

# # Define the total marks
# total_marks = 50

# # Define the criteria and corresponding marks allocation
# marks_criteria = {
# #     'Cosine_Value_Keywords': {'High': 15, 'Low': 10},
#     'CommentType3': {'Detail': 10, 'Basic': 5},
#     'Comment_Category': {'Both Error and Solution': 20, 'Solution Only': 10, 'Error Only': 10, 'None': 2},
#     'Total_Answer_Keyword2': {'>4': 20, '=4': 15, '<4': 10, '=0': 1}
# }
# teacher_keywords = ['variable', 'declaration', 'integer', 'int', 'declare','datatype',"data","type"]
# lenanskeyword= len(teacher_keywords)

# # Function to assign marks based on the criteria
# def assign_marks(row):
#     marks = 0
#     assignmarks=[]
#     for criterion, allocation in marks_criteria.items():
#         value = row[criterion]
# #         print(f"{value}-->{allocation}")
        
#         if value in allocation:
#             marks += allocation[value]
#             assignmarks.append(allocation[value])
#         elif value >= (lenanskeyword)*0.60:
# #             print(f"{value}-->{(lenanskeyword)*0.60}")
#             marks += 15
#             assignmarks.append(value)
#         elif value <= (lenanskeyword)*0.60:
# #             print(f"{value}-->{(lenanskeyword)*0.60}")
#             marks += 10
#             assignmarks.append(value)
#         elif value == 0:
# #             print(f"{value}-->{(lenanskeyword)*0.60}")
#             assignmarks.append(value)
#             marks += 0
#         else:
#              marks += 0  # If criterion not found, assign 0 marks
# #         print(assignmarks)
#     return marks
# # Apply the function to the DataFrame to calculate the marks for each comment
# newdata['Mark_With_3Rule'] = newdata.apply(assign_marks, axis=1)

# # Display the updated DataFrame with marks
# print(newdata[["ID","Name","Mark_With_3Rule"]])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming your data is loaded into a pandas DataFrame called 'data' with columns:
# 'comment_id', 'ML_mark', 'rule_based_mark'

# Calculate absolute difference in marks for highlighting discrepancies
comment_data['mark_difference'] = abs(comment_data['ML_mark'] - comment_data['rule_based_mark'])

# Define colors based on difference threshold (adjust threshold as needed)
colors = ['blue' if diff < 2 else 'red' for diff in data['mark_difference']]

# Create the scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(comment_data['ID'], comment_data['ML_mark'], label='ML Mark', color=colors)
plt.scatter(comment_data['ID'], comment_data['rule_based_mark'], label='Rule-Based Mark')

# Add trendline for ML marks
m, b = np.polyfit(comment_data['ID'], comment_data['ML_mark'], 1)
plt.plot(comment_data['ID'], m * comment_data['ID'] + b, color='green', linestyle='--', label='ML Trendline')

# Labels, title, and legend
plt.xlabel('Comment ID')
plt.ylabel('Mark')
plt.title('Comparison of Assigned Marks')
plt.legend()

# Highlight discrepancies in the legend (optional)
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.grid(True)
plt.show()


### <FONT COLOR="BLUE"> ASSIGN MARKS USING COSINE SIMILARITY </FONT>

#### <FONT COLOR="RED"> USING WORD2VEC</FONT>

In [ ]:
# import pandas as pd
# from gensim.models import Word2Vec  # Import Word2Vec if not already available
# from sklearn.metrics.pairwise import cosine_similarity

# # Sample teacher's answer (assuming it's the same for all comments)
# teacher_answer = "Forget to declare datatype of variable during declaration. So it generate syntax error. We have to declare variable integer because in printf statement it use %d as format specifier."

# # Load a pre-trained Word2Vec model (replace with the path to your model)
# model = Word2Vec.load("path/to/your/word2vec.model")

# def calculate_cosine_similarity(comment, model, teacher_keywords):
#   """
#   Calculates cosine similarity between a comment and teacher keywords.

#   Args:
#       comment: The student's comment (string).
#       model: The loaded Word2Vec model.
#       teacher_keywords: A list of teacher answer keywords (strings).

#   Returns:
#       The cosine similarity score between the comment and teacher keywords (float).
#   """
#   # Pre-process comment (optional: lowercase, remove punctuation)
#   comment = comment.lower()
#   comment_embedding = sentence_embedding(comment, model)

#   # Create a document vector representing teacher keywords (average of word vectors)
#   teacher_embedding = np.mean([model.wv[word] for word in teacher_keywords if word in model.wv.vocab], axis=0)

#   # Calculate cosine similarity
#   return cosine_similarity(comment_embedding.reshape(1, -1), teacher_embedding.reshape(1, -1))[0][0]

# # Function to generate sentence embedding (same as previous example)
# def sentence_embedding(sentence, model):
#   # ... (same implementation as before)

# # Assuming your DataFrame is called 'data' with a column 'cleancomment3'
# def assign_marks_with_cosine(data, teacher_answer, teacher_keywords):
#   """
#   Assigns marks based on cosine similarity and predefined thresholds.

#   Args:
#       data: The pandas DataFrame containing comments.
#       teacher_answer: The teacher's answer (string).
#       teacher_keywords: A list of teacher answer keywords (strings).

#   Returns:
#       A modified DataFrame with a new column 'Marks_With_Cosine' containing marks.
#   """
#   data['Marks_With_Cosine'] = data['cleancomment3'].apply(lambda comment: calculate_cosine_similarity(comment, model, teacher_keywords))
#   data['Marks_With_Cosine'] = data['Marks_With_Cosine'].apply(lambda score: 
#                                                          50 if score > 0.7 else (
#                                                              35 if score > 0.5 else (
#                                                                  30 if score > 0.2 else 10
#                                                              )
#                                                          )
#                                                        )
#   return data

# # Apply the function to your DataFrame
# data = assign_marks_with_cosine(data.copy(), teacher_answer, teacher_keywords)

# # Print the DataFrame with the new 'Marks_With_Cosine' column (optional)
# print(data)


#### <FONT COLOR="RED">USING COUNTVECTORIZE</FONT>

In [ ]:
def assign_marks_with_countvectorizer(data, threshold_high=0.7, threshold_medium=0.5, marks_high=50, marks_medium=35, marks_low=10):
  """
  Assigns marks based on cosine similarity with CountVectorizer and predefined thresholds.

  Args:
      data: The pandas DataFrame containing comments and "Cosine_Similarity_Keywords" column.
      threshold_high: Cosine similarity threshold for high marks (default 0.7).
      threshold_medium: Cosine similarity threshold for medium marks (default 0.5).
      marks_high: Marks awarded for high similarity (default 50).
      marks_medium: Marks awarded for medium similarity (default 35).
      marks_low: Marks awarded for low similarity (default 10).

  Returns:
      A modified DataFrame with a new column 'Marks_With_CV' containing marks.
  """
  data['Marks_With_Cosine'] = data['Cosine_Similarity_Keywords'].apply(lambda score: 
                                                         marks_high if score > threshold_high else (
                                                             marks_medium if score > threshold_medium else marks_low
                                                         )
                                                       )
  return data

# Assuming your DataFrame is called 'data'
newdata = assign_marks_with_countvectorizer(newdata.copy())

# Print the DataFrame with the new 'Marks_With_CV' column (optional)
print(newdata)


### <font color="red"> FIND COSINE USING TF-IDF</font>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Assuming 'teacher_keywords' is a list of teacher answer keywords and 'student_comments' is a list of student comments
teacher_keywords = ['variable', 'declaration', 'integer', 'int', 'declare', 'datatype', "data", "type"]
student_comments = newdata["CorrectComment2_Spell"]

# Create TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the teacher keywords and transform the student comments using the TF-IDF vectorizer
tfidf_matrix_keywords = tfidf_vectorizer.fit_transform([" ".join(teacher_keywords)])
tfidf_matrix_comments = tfidf_vectorizer.transform(student_comments)

# Calculate cosine similarity between teacher keywords and student comments
cosine_similarities = cosine_similarity(tfidf_matrix_keywords, tfidf_matrix_comments)

# Store cosine similarity values in newdata dataframe
newdata['Cosine_Similarity_Tfidf_Keywords'] = cosine_similarities.squeeze()

# Print ID, Cosine_Similarity_Keywords, and Cosine_Similarity_Tfidf_Keywords
# print(newdata[['ID', 'Cosine_Similarity_Keywords']])


In [ ]:
newdata[["ID","Cosine_Similarity_Keywords",'Cosine_Similarity_Tfidf_Keywords']]

In [ ]:
newdata[['Match_with_Answer_keywords2', 'Match_with_Answer_keywords3','Total_Answer_Keyword2']]

### <font color="Blue"> APPLY SENTIMENT ANALYSIS </font> [ NOT HELPFUL ]

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Download the VADER lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
sid = SentimentIntensityAnalyzer()

# Perform sentiment analysis on each comment
sentiment_scores = []
for comment in newdata['CorrectComment3_Spell']:
    # Calculate sentiment scores
    scores = sid.polarity_scores(comment)
    sentiment_scores.append(scores)

# Extract compound scores
compound_scores = [score['compound'] for score in sentiment_scores]

# Classify comments based on sentiment scores
classification = []
for score in compound_scores:
    if score > 0.05:  # Positive sentiment
        classification.append("Solution Only")
    elif score < -0.05:  # Negative sentiment
        classification.append("Error Only")
    else:  # Neutral sentiment
        classification.append("Both Error and Solution")

# Add sentiment scores and classification to the DataFrame
newdata['Sentiment_Score'] = compound_scores
newdata['Sentiment_Classification'] = classification

# Display the DataFrame with sentiment scores and classification
print(newdata[['ID','CorrectComment3_Spell', 'Sentiment_Score', 'Sentiment_Classification']])


#### SENTIMENT ANALYSIS : it is not much helpful to find comment category

### <font color="blue"> CREATE MODEL AND APPLY ML ALGORITHM AND CHECK ACCURACY </FONT>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Assuming 'newdata' is your DataFrame containing the dataset with relevant features

# Step 1: Prepare the data

# Step 2: Create the target variable 'Mark_With_3Rule' based on features
# Assuming 'Mark_With_3Rule' is already calculated based on rule-based method

# Step 3: Preprocess the data for the 3-rule method
encoder_3rule = OneHotEncoder(drop='first', sparse=False)
X_encoded_3rule = encoder_3rule.fit_transform(newdata[['CommentType3', 'Comment_Category']].values)
X_processed_3rule = np.concatenate([X_encoded_3rule, newdata[['Cosine_Similarity_Keywords', 'Total_Answer_Keyword2']].values], axis=1)
y_3rule = newdata['Mark_With_3Rule']

# Step 4: Split the data into training and testing sets for the 3-rule method
X_train_3rule, X_test_3rule, y_train_3rule, y_test_3rule = train_test_split(X_processed_3rule, y_3rule, test_size=0.2, random_state=42)

# Step 5: Define models for the 3-rule method
models_3rule = {
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor()
}

# Step 6: Train and evaluate models for the 3-rule method
results_3rule = {}
for name, model in models_3rule.items():
    model.fit(X_train_3rule, y_train_3rule)
    y_pred_3rule = model.predict(X_test_3rule)
    mse_3rule = mean_squared_error(y_test_3rule, y_pred_3rule)
    results_3rule[name] = mse_3rule

# Step 7: Select the best model for the 3-rule method
best_model_name_3rule = min(results_3rule, key=results_3rule.get)
best_model_3rule = models_3rule[best_model_name_3rule]
print("Best Model for 3-rule method:", best_model_name_3rule)
print("Mean Squared Error on Test Set (3-rule method):", results_3rule[best_model_name_3rule])

# Repeat Steps 3-7 for the cosine method

# Step 3: Preprocess the data for the cosine method
# Assuming 'Mark_With_Cosine' is already calculated based on cosine similarity method

# Step 4: Split the data into training and testing sets for the cosine method

# Step 5: Define models for the cosine method

# Step 6: Train and evaluate models for the cosine method

# Step 7: Select the best model for the cosine method

# Predict marks using the best model for the 3-rule method
best_model_3rule.fit(X_processed_3rule, y_3rule)
y_pred_3rule_best = best_model_3rule.predict(X_processed_3rule)
newdata['Mark_With_ML_3Rule'] = y_pred_3rule_best

# Repeat the prediction process for the cosine method

# Display the results
print("Best Model for 3-rule method:", best_model_name_3rule)
print("Mean Squared Error on Test Set (3-rule method):", results_3rule[best_model_name_3rule])
# Repeat for the cosine method



In [ ]:
newdata[['Mark_With_3Rule', 'Marks_With_Cosine', 'Mark_With_ML_3Rule']]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Select relevant features and target variable
X = newdata[['Comment_Length','Cosine_Similarity_Keywords','Total_Answer_Keyword2']]
y = newdata['Mark_With_3Rule']

# Encode categorical variables
label_encoder = LabelEncoder()
X['CommentType'] = label_encoder.fit_transform(newdata['Comment_Length'] > 50)
X['CommentCategory'] = label_encoder.fit_transform(newdata['Comment_Category'])

# Derive Total_Matching_Keywords
# Convert columns to string and then split them to compute the total number of keywords
# keyword_columns = ['Error_Matched_Keywords3', 'Solution_Matched_Keywords3', 'Both_Matched_Keywords3']
# X['Total_Matching_Keywords'] = newdata[keyword_columns].astype(str).apply(lambda x: x.str.split(',').str.len()).sum(axis=1)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'Support Vector Machines': SVC()
}

# Train and evaluate models
results = {}
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy

# Compare results
for name, accuracy in results.items():
    print(f"{name}: Accuracy = {accuracy:.4f}")

best_model = max(results, key=results.get)
print(f"\nBest Model: {best_model}")


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

# Step 1: Prepare the data
# Assuming 'newdata' is your DataFrame containing the dataset with relevant features

# Step 2: Encode categorical variables using one-hot encoding
encoder = OneHotEncoder()
encoded_data = encoder.fit_transform(newdata[['CommentType3', 'Comment_Category']])

# Combine encoded features with numerical features
X = np.concatenate([encoded_data.toarray(), newdata[['Cosine_Similarity_Keywords', 'Total_Answer_Keyword2']].values], axis=1)

# Target variable for rule-based method
y_3rule = newdata['Mark_With_3Rule']

# Target variable for cosine similarity method
y_cosine = newdata['Marks_With_Cosine']

# Split the data into training and testing sets for 3-rule method
X_train_3rule, X_test_3rule, y_train_3rule, y_test_3rule = train_test_split(X, y_3rule, test_size=0.2, random_state=42)

# Split the data into training and testing sets for cosine similarity method
X_train_cosine, X_test_cosine, y_train_cosine, y_test_cosine = train_test_split(X, y_cosine, test_size=0.2, random_state=42)

# Step 3: Define models
models = {
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "Support Vector Machine": SVR(),
    "Linear Regression": LinearRegression()
}

# Step 4: Train and evaluate models for 3-rule method
results_3rule = {}
for name, model in models.items():
    # Train the model
    model.fit(X_train_3rule, y_train_3rule)
    # Evaluate model performance
    y_pred_3rule = model.predict(X_test_3rule)
    mse_3rule = mean_squared_error(y_test_3rule, y_pred_3rule)
    results_3rule[name] = mse_3rule

# Step 5: Train and evaluate models for cosine similarity method
results_cosine = {}
for name, model in models.items():
    # Train the model
    model.fit(X_train_cosine, y_train_cosine)
    # Evaluate model performance
    y_pred_cosine = model.predict(X_test_cosine)
    mse_cosine = mean_squared_error(y_test_cosine, y_pred_cosine)
    results_cosine[name] = mse_cosine

# Step 6: Select the best model for 3-rule method
best_model_3rule_name = min(results_3rule, key=results_3rule.get)
best_model_3rule = models[best_model_3rule_name]
print("Best Model for 3-rule method:", best_model_3rule_name)
print("Mean Squared Error on Test Set for 3-rule method:", results_3rule[best_model_3rule_name])

# Step 7: Select the best model for cosine similarity method
best_model_cosine_name = min(results_cosine, key=results_cosine.get)
best_model_cosine = models[best_model_cosine_name]
print("Best Model for cosine similarity method:", best_model_cosine_name)
print("Mean Squared Error on Test Set for cosine similarity method:", results_cosine[best_model_cosine_name])

# Step 8: Predict marks using the best models and store them in new columns
# best_model_3rule.fit(X, y_3rule)
# best_model_cosine.fit(X, y_cosine)

# newdata['Mark_With_ML_3Rule'] = best_model_3rule.predict(X)
# newdata['Mark_With_ML_Cosine'] = best_model_cosine.predict(X)
# Step 8: Predict marks using all models and store them in new columns
for name, model in models.items():
    newdata[f'Mark_With_ML_{name.replace(" ", "_")}'] = model.predict(X)

# Print the marks predicted by all algorithms
for name, model in models.items():
    predicted_marks = newdata[f'Mark_With_ML_{name.replace(" ", "_")}']
    print(f"Predicted marks by {name}: {predicted_marks}")




In [ ]:
newdata.columns

In [ ]:
newdata[['Mark_With_4Rule',
       'Mark_With_3Rule', 'Marks_With_Cosine', 'Mark_With_ML_3Rule',
       'Mark_With_ML_Cosine', 'Mark_With_ML_Random_Forest',
       'Mark_With_ML_Gradient_Boosting', 'Mark_With_ML_Support_Vector_Machine',
       'Mark_With_ML_Linear_Regression']]

- Based on above result , we said that "Random Forest" algorithm gives best result.

### <FONT COLOR="BLUE"> APPLY ML USING RULE BASED METHOD MARKS</FONT>

In [ ]:
# Step 1: Prepare the data
# Assuming 'newdata' is your DataFrame containing the dataset with relevant features

# Step 2: Create the target variable 'Mark' based on the rule-based method
# For demonstration purposes, let's say you assign marks based on some logic
# Replace this logic with your actual method to assign marks
newdata['Mark_With_Rule'] = ...  # Assign marks based on the rule-based method

# Step 3: Define the features and target variable
X = newdata[['Cosine_Similarity_Keywords', 'CommentType3', 'Total_Answer_Keyword2', 'Comment_Category']]
y = newdata['Mark_With_3Rule']

# Step 4: Encode categorical variables using one-hot encoding
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first', sparse=False)
X_encoded = encoder.fit_transform(X[['CommentType3', 'Comment_Category']])

# Combine encoded features with numerical features
X_processed = np.concatenate([X_encoded, X.drop(columns=['CommentType3', 'Comment_Category']).values], axis=1)

# Step 5: Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# Step 6: Define ML models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

models = {
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "Linear Regression": LinearRegression(),
    "SVR": SVR()
}

# Step 7: Train and evaluate ML models
from sklearn.metrics import mean_squared_error

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    results[name] = mse

# Step 8: Select the best model
best_model_name = min(results, key=results.get)
best_model = models[best_model_name]
print("Best Model for Rule-Based Method:", best_model_name)
print("Mean Squared Error on Test Set for Rule-Based Method:", results[best_model_name])




In [ ]:
# Step 1: Prepare the data
# Assuming 'newdata' is your DataFrame containing the dataset with relevant features

# Step 2: Create the target variable 'Mark' based on the rule-based method
# For demonstration purposes, let's say you assign marks based on some logic
# Replace this logic with your actual method to assign marks
newdata['Mark_With_Rule'] = ...  # Assign marks based on the rule-based method

# Step 3: Define the features and target variable
X = newdata[['Cosine_Similarity_Keywords', 'CommentType3', 'Total_Answer_Keyword2', 'Comment_Category']]
y = newdata['Mark_With_3Rule']

# Step 4: Encode categorical variables using one-hot encoding
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first', sparse=False)
X_encoded = encoder.fit_transform(X[['CommentType3', 'Comment_Category']])

# Combine encoded features with numerical features
X_processed = np.concatenate([X_encoded, X.drop(columns=['CommentType3', 'Comment_Category']).values], axis=1)

# Step 5: Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# Step 6: Define ML models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

models = {
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "Linear Regression": LinearRegression(),
    "SVR": SVR()
}

# Step 7: Train and evaluate ML models
from sklearn.metrics import mean_squared_error

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    results[name] = mse



# Step 8: Train and evaluate ML models
from sklearn.metrics import mean_squared_error

results = {}
predicted_marks = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    results[name] = mse
    predicted_marks[name] = y_pred

# Step 9: Print the predicted marks of all algorithms
print("Predicted Marks by Rule-Based Method:")
print(y_test.values)  # Actual marks from the test set
print("\nPredicted Marks by ML Algorithms:")
for name, marks in predicted_marks.items():
    print(f"{name}: {marks}")

# Step 10: Select the best model
best_model_name = min(results, key=results.get)
best_model = models[best_model_name]
print("\nBest Model for Rule-Based Method:", best_model_name)
print("Mean Squared Error on Test Set for Rule-Based Method:", results[best_model_name])


### <FONT COLOR="BLUE"> APPLY RANDOM FOREST AND ASSIGN MARK </FONT>

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Select relevant features and target variable
X = newdata[['Cosine_Similarity_Keywords', 'CommentType3', 'Total_Answer_Keyword2', 'Comment_Category']]
y = newdata['Mark_With_3Rule']

# Step 4: Encode categorical variables using one-hot encoding
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first', sparse=False)
X_encoded = encoder.fit_transform(X[['CommentType3', 'Comment_Category']])

# Combine encoded features with numerical features
X_processed = np.concatenate([X_encoded, X.drop(columns=['CommentType3', 'Comment_Category']).values], axis=1)

# # Scale numerical features
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# Train Random Forest model on the entire dataset
rf_model = RandomForestRegressor()
rf_model.fit(X_processed, y)

# Predict marks for each student
predicted_marks = rf_model.predict(X_processed)

# Add predicted marks to the DataFrame
newdata['Predicted_Marks'] = predicted_marks

# Display the DataFrame with predicted marks
print(newdata[["ID",'Name', 'Mark_With_3Rule','Predicted_Marks']])








In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.preprocessing import LabelEncoder, StandardScaler

# # Select relevant features and target variable
# X = newdata[['Cosine_Similarity_Keywords', 'CommentType3', 'Total_Answer_Keyword2', 'Comment_Category']]
# y = newdata['Marks_With_Cosine']

# # Step 4: Encode categorical variables using one-hot encoding
# from sklearn.preprocessing import OneHotEncoder
# encoder = OneHotEncoder(drop='first', sparse=False)
# X_encoded = encoder.fit_transform(X[['CommentType3', 'Comment_Category']])

# # Combine encoded features with numerical features
# X_processed = np.concatenate([X_encoded, X.drop(columns=['CommentType3', 'Comment_Category']).values], axis=1)

# # # Scale numerical features
# # scaler = StandardScaler()
# # X_scaled = scaler.fit_transform(X)

# # Train Random Forest model on the entire dataset
# rf_model = RandomForestRegressor()
# rf_model.fit(X_processed, y)

# # Predict marks for each student
# predicted_marks = rf_model.predict(X_processed)

# # Add predicted marks to the DataFrame
# newdata['Predicted_Marks_Cos_RF'] = predicted_marks

# # Display the DataFrame with predicted marks
# # print(newdata[["ID",'Name', 'Mark_With_3Rule','Marks_With_Cosine','Predicted_Marks']])


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Select relevant features and target variable
X = newdata[['Cosine_Similarity_Keywords', 'CommentType3', 'Total_Answer_Keyword2', 'Comment_Category']]
y_cosine = newdata['Marks_With_Cosine']

# Step 4: Encode categorical variables using one-hot encoding
encoder = OneHotEncoder(drop='first', sparse=False)
X_encoded = encoder.fit_transform(X[['CommentType3', 'Comment_Category']])

# Combine encoded features with numerical features
X_processed = np.concatenate([X_encoded, X.drop(columns=['CommentType3', 'Comment_Category']).values], axis=1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, y_cosine, test_size=0.2, random_state=42)

# Train Random Forest model on the training set
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

# Predict marks for the testing set
predicted_marks_rf = rf_model.predict(X_test)
# newdata['predicted_marks_RF']=predicted_marks_rf
# Compare the predicted marks with the marks generated by cosine similarity
comparison_df = pd.DataFrame({'Predicted_Marks_RF': predicted_marks_rf, 'Marks_With_Cosine': y_test})
print(comparison_df)
print(len(comparison_df))

# Extract true marks from the DataFrame
y_true = newdata['Marks_With_Cosine']

# Compute ML-generated marks using the trained model
predicted_marks_ml = rf_model.predict(X_processed)

# Compare ML-generated marks with cosine similarity-generated marks
mse_ml = mean_squared_error(y_true, predicted_marks_ml)
mae_ml = mean_absolute_error(y_true, predicted_marks_ml)
r2_ml = r2_score(y_true, predicted_marks_ml)

# Print evaluation metrics
print("Mean Squared Error (ML):", mse_ml)
print("Mean Absolute Error (ML):", mae_ml)
print("R^2 Score (ML):", r2_ml)





### CHECK ACCURACY

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Step 1: Prepare the data
# Extract features and target variables from rule-based method
X_rule = newdata[['CommentType3', 'Total_Answer_Keyword2', 'Comment_Category']]
y_rule = newdata['Mark_With_3Rule']

# Extract features and target variables from cosine similarity method
X_cosine = newdata[['Cosine_Similarity_Keywords', 'CommentType3', 'Total_Answer_Keyword2', 'Comment_Category']]
y_cosine = newdata['Marks_With_Cosine']

# Step 2: Combine features from both methods
# Rule-based method features
encoder = OneHotEncoder(drop='first', sparse=False)
X_rule_encoded = encoder.fit_transform(X_rule[['CommentType3', 'Comment_Category']])
X_rule_processed = np.concatenate([X_rule_encoded, X_rule.drop(columns=['CommentType3', 'Comment_Category'])], axis=1)

# Cosine similarity method features
X_cosine_encoded = encoder.transform(X_cosine[['CommentType3', 'Comment_Category']])
X_cosine_processed = np.concatenate([X_cosine_encoded, X_cosine.drop(columns=['CommentType3', 'Comment_Category'])], axis=1)

# Combine features from both methods
X_combined = np.concatenate([X_rule_processed, X_cosine_processed], axis=1)
y_combined = y_rule  # Use marks from the rule-based method as target variable for training

# Step 3: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_combined, test_size=0.2, random_state=42)

# Step 4: Train a Random Forest regressor
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

# Step 5: Predict marks using the trained model
predicted_marks = rf_model.predict(X_test)

# Step 6: Print actual and predicted marks
print("Actual Marks:", y_test,"Predicted Marks:", predicted_marks)

# Step 7: Evaluate the performance of the model
mse = mean_squared_error(y_test, predicted_marks)
mae = mean_absolute_error(y_test, predicted_marks)
r2 = r2_score(y_test, predicted_marks)

print("Mean Squared Error (ML):", mse)
print("Mean Absolute Error (ML):", mae)
print("R^2 Score (ML):", r2)


In [ ]:
newdata[["ID",'Name', 'Mark_With_3Rule','Predicted_Marks','Marks_With_Cosine','Predicted_Marks_Cos_RF']]

### <FONT COLOR="BLUE">FIND THE DIFFERENCE BETWEEN MARK AND RANDOM FOREST PREDICTED MARKS </FONT>

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Select relevant features and target variable
X = newdata[['Cosine_Similarity_Keywords', 'CommentType3', 'Total_Answer_Keyword2', 'Comment_Category']]
y = newdata['Mark_With_3Rule']


# Encode categorical variables
label_encoder = LabelEncoder()
X['CommentType3'] = label_encoder.fit_transform(X['CommentType3'])
X['Comment_Category'] = label_encoder.fit_transform(X['Comment_Category'])
# Scale numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train Random Forest model on the entire dataset
rf_model = RandomForestRegressor()
rf_model.fit(X_scaled, y)

# Predict marks for each student
predicted_marks = rf_model.predict(X_scaled)

# Add predicted marks to the DataFrame
newdata['Predicted_Marks'] = predicted_marks

# Compare predicted marks with manually calculated marks
newdata['Difference'] = newdata['Mark_With_3Rule'] - newdata['Predicted_Marks']

# Write DataFrame to CSV file
csv_file_path = 'G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\predicted_marks_comparison.csv'
newdata.to_csv(csv_file_path, index=False)

# Display the DataFrame with predicted marks and difference
print(newdata[['Name', 'Mark_With_3Rule', 'Predicted_Marks', 'Difference']])


### <font color="blue"> APPLY GRADIENT BOOSTING ALGORITHM </FONT>

In [ ]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score



# Select features and target variable
X = newdata[['Cosine_Similarity_Keywords', 'CommentType3', 'Total_Answer_Keyword2', 'Comment_Category']]
y = newdata['Mark_With_3Rule']

# Encode categorical variables
label_encoder = LabelEncoder()
X['CommentType3'] = label_encoder.fit_transform(X['CommentType3'])
X['Comment_Category'] = label_encoder.fit_transform(X['Comment_Category'])
# Scale numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Gradient Boosting Regressor
gb_model = GradientBoostingRegressor()

# Train the model
gb_model.fit(X_train, y_train)

# Predict marks on the test set
y_pred = gb_model.predict(X_test)

# Print the predicted marks
print("Predicted Marks with Gradient Boosting Algorithm:")
for i, pred_mark in enumerate(y_pred):
    print(f"Student {i+1}: Predicted Mark = {pred_mark}")

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nAccuracy Metrics:")
print("Mean Squared Error (GB):", mse)
print("Mean Absolute Error (GB):", mae)
print("R^2 Score (GB):", r2)


In [ ]:
data.columns

In [ ]:
newdata[["CleanComment3_Length",'CleanComment4_Length']]=data[["CleanComment3_Length",'CleanComment4_Length']].copy()

In [ ]:
newdata.columns

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

# Select relevant features and target variable
X = newdata[['Comment_Length', 'Cosine_Similarity_Keywords', 'CommentType3', 'Total_Answer_Keyword2']]
y = newdata['Mark']

# Encode categorical variables
label_encoder = LabelEncoder()
X['CommentType3'] = label_encoder.fit_transform(X['CommentType3'])

# Scale numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train Random Forest model on the entire dataset
rf_model = RandomForestRegressor()
rf_model.fit(X_scaled, y)

# Predict marks for each student
predicted_marks = rf_model.predict(X_scaled)

# Take the ceiling value of predicted marks
ceiling_predicted_marks = np.ceil(predicted_marks)

# Add predicted marks to the DataFrame
newdata['Predicted_Marks'] = ceiling_predicted_marks

# Compare predicted marks with manually calculated marks
newdata['Difference'] = newdata['Mark'] - newdata['Predicted_Marks']

# Write DataFrame to CSV file
csv_file_path = 'G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\predicted_marks_comparison.csv'
newdata.to_csv(csv_file_path, index=False)

# Display the DataFrame with predicted marks and difference
print(newdata[['Name', 'Mark', 'Predicted_Marks', 'Difference']])


### FIND ACCURACY BETWEEN MARKS

In [ ]:
# Define a threshold for considering a prediction as correct
threshold = 1

# Calculate the absolute difference between Mark and Predicted_Marks
newdata['Absolute_Difference'] = abs(newdata['Mark_With_3Rule'] - newdata['Predicted_Marks'])

# Count the number of correct predictions
correct_predictions = (newdata['Absolute_Difference'] <= threshold).sum()

# Calculate the accuracy
total_samples = len(newdata)
accuracy = correct_predictions / total_samples

print("Accuracy:", accuracy)


### <FONT COLOR="BLUE"> APPLY SVM & REGRESSION MODEL TO PREDICT MARK </FONT>

In [ ]:
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression

# Train SVM model
svm_model = SVR()
svm_model.fit(X_scaled, y)

# Predict marks using SVM
svm_predicted_marks = svm_model.predict(X_scaled)

# Train Regression model
regression_model = LinearRegression()
regression_model.fit(X_scaled, y)

# Predict marks using Regression
regression_predicted_marks = regression_model.predict(X_scaled)

# Add predicted marks to the DataFrame
newdata['SVM_Predicted_Marks'] = svm_predicted_marks
newdata['Regression_Predicted_Marks'] = regression_predicted_marks

# Calculate the accuracy for each model
threshold = 1
newdata['SVM_Absolute_Difference'] = abs(newdata['Mark'] - newdata['SVM_Predicted_Marks'])
newdata['Regression_Absolute_Difference'] = abs(newdata['Mark'] - newdata['Regression_Predicted_Marks'])

svm_correct_predictions = (newdata['SVM_Absolute_Difference'] <= threshold).sum()
regression_correct_predictions = (newdata['Regression_Absolute_Difference'] <= threshold).sum()

svm_accuracy = svm_correct_predictions / total_samples
regression_accuracy = regression_correct_predictions / total_samples

print("SVM Accuracy:", svm_accuracy)
print("Regression Accuracy:", regression_accuracy)


In [ ]:
newdata[['Mark', 'Predicted_Marks', 'SVM_Predicted_Marks', 'Regression_Predicted_Marks']]


### <FONT COLOR="BLUE"> FIND ACCURACY BASED ON MARKS </FONT>

In [ ]:
# Define a function to calculate accuracy based on absolute difference
def calculate_accuracy(true_marks, predicted_marks):
    threshold = 1  # Define the threshold for considering a prediction as correct
    absolute_difference = abs(true_marks - predicted_marks)
    correct_predictions = (absolute_difference <= threshold).sum()
    total_samples = len(true_marks)
    accuracy = correct_predictions / total_samples
    return accuracy

# Calculate accuracy for Random Forest
rf_accuracy = calculate_accuracy(newdata['Mark'], newdata['Predicted_Marks'])

# Calculate accuracy for SVM
svm_accuracy = calculate_accuracy(newdata['Mark'], newdata['SVM_Predicted_Marks'])

# Calculate accuracy for Regression
regression_accuracy = calculate_accuracy(newdata['Mark'], newdata['Regression_Predicted_Marks'])

# Print accuracy values
print("Accuracy of Random Forest:", rf_accuracy)
print("Accuracy of SVM:", svm_accuracy)
print("Accuracy of Regression:", regression_accuracy)


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Define a function to calculate accuracy based on absolute difference
def calculate_accuracy(true_marks, predicted_marks):
    threshold = 1  # Define the threshold for considering a prediction as correct
    absolute_difference = abs(true_marks - predicted_marks)
    correct_predictions = (absolute_difference <= threshold).sum()
    total_samples = len(true_marks)
    accuracy = correct_predictions / total_samples
    return accuracy

# Calculate accuracy for Random Forest
rf_accuracy = calculate_accuracy(newdata['Mark'], newdata['Predicted_Marks'])

# Calculate other evaluation metrics for Random Forest
rf_precision = precision_score(newdata['Mark'], newdata['Predicted_Marks'], average='macro')
rf_recall = recall_score(newdata['Mark'], newdata['Predicted_Marks'], average='macro')
rf_f1 = f1_score(newdata['Mark'], newdata['Predicted_Marks'], average='macro')

# Print evaluation metrics for Random Forest
print("Evaluation metrics for Random Forest:")
print("Accuracy:", rf_accuracy)
print("Precision:", rf_precision)
print("Recall:", rf_recall)
print("F1 Score:", rf_f1)

# Similarly, calculate evaluation metrics for SVM and Regression models
# Modify the code accordingly for SVM and Regression models



### <font color="blue"> FIND SIMILARITY USING JACCARD, BERT AND OTHER METHOD </FONT>

In [ ]:
newdata.columns

### <font color="Blue"> APPLY ML ALGORITHM TO FIND COMMENT IS BASIC OR DETAIL </FONT>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load the labeled dataset
data = pd.read_csv("G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\predicted_marks_comparison.csv")  # Assuming you have a CSV file with labeled data

# Prepare features and target variable
X = data[['CleanComment3_Length']]  # Assuming 'Comment_Length' is a column containing comment lengths
y = data['CommentType3']  # Assuming 'CommentType' is a column containing comment types (basic or detail)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Logistic Regression model
model_comment_type = LogisticRegression()
model_comment_type.fit(X_train, y_train)

# Make predictions
y_pred = model_comment_type.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Store predictions in a new column
data['Pred_CommentType'] = model_comment_type.predict(data[['CleanComment3_Length']])
print((data["Pred_CommentType"]))
print(len(data))

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# Load the labeled dataset
data = pd.read_csv("G:\\GitHub Classroom Assistant\\Topic Modelling Dataset\\TMC\\predicted_marks_comparison.csv")  # Assuming you have a CSV file with labeled data

# Prepare features and target variable
X = data[['CleanComment3_Length']]  # Assuming 'Comment_Length' is a column containing comment lengths
y = data['CommentType3']  # Assuming 'CommentType' is a column containing comment types (basic or detail)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate multiple models
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy:", accuracy)

In [ ]:
data.info()

In [ ]:
data[["ID","CleanComment3","CommentType3",'Pred_CommentType']]

In [ ]:
# Create a DataFrame with ID, clean comment, actual comment type, and predicted comment type
results = data[['ID', 'CleanComment3', 'CommentType3', 'Pred_CommentType']]

# Filter out rows where the predicted comment type does not match the actual comment type
incorrect_predictions = results[results['CommentType3'] != results['Pred_CommentType']]

# Print the DataFrame containing incorrect predictions
print(incorrect_predictions)


In [ ]:
data['Pred_CommentType'].value_counts()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Step 1: Split the dataset into features (X) and target variable (y)
X = newdata[['CleanComment3_Length', 'Total_Answer_Keyword2', 'Cosine_Similarity_Keywords', 'Comment_Category']]
y = newdata['Mark']

# Step 2: Encode categorical variable 'Comment_Category' using one-hot encoding
encoder = OneHotEncoder()
X_cat = encoder.fit_transform(newdata[['Comment_Category']])

# Step 3: Combine encoded categorical variable and other numerical features
X_numerical = newdata[['CleanComment3_Length', 'Total_Answer_Keyword2', 'Cosine_Similarity_Keywords']]
X = np.concatenate([X_numerical, X_cat.toarray()], axis=1)

# Step 4: Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Train the machine learning model on the training set
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 6: Evaluate the model on the validation set
y_pred = model.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
print("Mean Squared Error on Validation Set:", mse)


In [ ]:
# Step 1: Calculate baseline prediction (mean or median)
baseline_prediction = y_train.mean()  # Or use y_train.median() for median

# Step 2: Create an array of baseline predictions for validation set
baseline_predictions_val = np.full_like(y_val, fill_value=baseline_prediction)

# Step 3: Calculate Mean Squared Error for baseline predictions
baseline_mse = mean_squared_error(y_val, baseline_predictions_val)

# Step 4: Compare baseline MSE with model MSE
print("Baseline Mean Squared Error:", baseline_mse)
print("Model Mean Squared Error:", mse)  # This is the MSE you calculated previously


In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Sample categorical data
categories = ['red', 'blue', 'green', 'red', 'green']

# Create an instance of OneHotEncoder
encoder = OneHotEncoder()

# Fit and transform the categorical data
encoded_data = encoder.fit_transform(np.array(categories).reshape(-1, 1))

# Convert the encoded data to a dense array
encoded_data_dense = encoded_data.toarray()

# Print the encoded data
print(encoded_data_dense)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Assuming X contains input features (including precomputed features) and y contains target variable (Mark)
# X should be a DataFrame containing features, and y should be a Series containing the target variable
# You can use train_test_split to split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the machine learning model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions on the validation set
y_pred = model.predict(X_val)

# Evaluate the model using Mean Squared Error (MSE)
mse = mean_squared_error(y_val, y_pred)
print("Mean Squared Error on Validation Set:", mse)

print("Fail to reject the null hypothesis: There is no significant difference between the marks.")



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

# Assuming X contains input features (including precomputed features) and y contains target variable (Mark)
# X should be a DataFrame containing features, and y should be a Series containing the target variable

# Step 1: Split the dataset into features (X) and target variable (y)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Initialize models
models = {
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "Linear Regression": LinearRegression(),
    "Support Vector Machine": SVR(),
    "K-Nearest Neighbors": KNeighborsRegressor()
}

# Step 3: Train and evaluate each model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    mse = mean_squared_error(y_val, y_pred)
    print(f"{name} Mean Squared Error on Validation Set: {mse}")


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

# Step 3: Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Initialize models
models = {
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "Linear Regression": LinearRegression(),
    "Support Vector Machine": SVR(),
    "K-Nearest Neighbors": KNeighborsRegressor()
}

# Step 5: Train and evaluate each model
for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict marks for the validation set
    y_pred = model.predict(X_val)
    
    # Store predicted marks in a separate column
    X_val[f"{name}_Predict_Mark"] = y_pred

    # Evaluate model performance (optional)
    mse = mean_squared_error(y_val, y_pred)
    print(f"{name} Mean Squared Error on Validation Set: {mse}")

# Optionally, you can save the modified DataFrame with predicted marks to a CSV file
# X_val.to_csv('predicted_marks.csv', index=False)
